In [1]:
import torchvision.transforms as transforms
import torch

from src.concept_bottleneck.dataset import CUB200_2011

# According to: https://pytorch.org/hub/pytorch_vision_inception_v3/
preprocess = transforms.Compose(
    [
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ]
)

training_data: CUB200_2011[torch.Tensor, torch.Tensor] = CUB200_2011(
    train=True, transform=preprocess, target_transform=torch.from_numpy  # type: ignore
)
test_data: CUB200_2011[torch.Tensor, torch.Tensor] = CUB200_2011(
    train=False, download=False, transform=preprocess, target_transform=torch.from_numpy  # type: ignore
)


Using downloaded and verified file: /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz
Extracting /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz to /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data


In [2]:
from torch.utils.data import DataLoader

batch_size = 16
num_workers = 2

train_dataloader = DataLoader(
    training_data, batch_size=batch_size, shuffle=True, num_workers=2
)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=2)


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model: torch.nn.Module = torch.hub.load(
    "pytorch/vision:v0.10.0",
    "inception_v3",
    init_weights=False,
    num_classes=training_data.num_attributes,
)
model = model.to(device)


Using cuda device


Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0


In [4]:
def train(
    dataloader: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
):
    size = len(dataloader.dataset)  # type: ignore
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(torch.float).to(device)  # type: ignore

        logits, aux_logits = model(X)
        loss = loss_fn(logits, y) + 0.4 * loss_fn(aux_logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [5]:
def test(
    dataloader: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
):
    size = len(dataloader.dataset)  # type: ignore
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        sigmoid = torch.nn.Sigmoid()
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # type: ignore
            logits = model(X)
            test_loss += loss_fn(logits, y.to(torch.float)).item()
            correct_attributes = torch.sum(
                ((sigmoid(logits) > 0.5).to(torch.int64) == y).to(torch.float)
            )
            num_attributes = y.shape[1]
            correct += correct_attributes.item() / num_attributes
    test_loss /= num_batches
    accuracy = correct / size
    return test_loss, accuracy


In [6]:
def save_model(model: torch.nn.Module, filename: str):
    torch.save(model.state_dict(), f"{filename}.pth")
    print("Saved PyTorch Model State to model.pth")


def load_model(model: torch.nn.Module, filename: str):
    model.load_state_dict(torch.load(f"{filename}.pth"))
    print("Loaded PyTorch Model State from model.pth")
    model.eval()


In [7]:
def run_epoch(epochs: int):
    loss_fn = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    train_losses: list[float] = []
    train_accuracies: list[float] = []
    test_losses: list[float] = []
    test_accuracies: list[float] = []
    for t in range(epochs):
        print(f"\nEpoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        train_loss, train_accuracy = test(train_dataloader, model, loss_fn)
        print(
            f"Train Accuracy: {(100 * train_accuracy):>0.10f}%, Avg loss: {train_loss:>8f}"
        )
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        test_loss, test_accuracy = test(test_dataloader, model, loss_fn)
        print(
            f"Test Accuracy: {(100 * test_accuracy):>0.10f}%, Avg loss: {test_loss:>8f}"
        )
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        if test_accuracy > 0.85 and t % 5 == 0:
            save_model(model, f"model_{t}")

        if test_accuracy > 0.98:
            print("Reached 98% accuracy so cancelling training")
            break

    print("Done!")
    return train_losses, train_accuracies, test_losses, test_accuracies


In [8]:
train_losses, train_accuracies, test_losses, test_accuracies = run_epoch(epochs=1000)
save_model(model, "model_final")

import json

with open("train_losses.json", "w") as f:
    json.dump(train_losses, f)

with open("train_accuracies.json", "w") as f:
    json.dump(train_accuracies, f)

with open("test_losses.json", "w") as f:
    json.dump(test_losses, f)

with open("test_accuracies.json", "w") as f:
    json.dump(test_accuracies, f)


Epoch 1
-------------------------------
loss: 0.995248  [    0/ 5994]
loss: 0.982231  [ 1600/ 5994]
loss: 0.963741  [ 3200/ 5994]
loss: 0.949623  [ 4800/ 5994]
Train Accuracy: 68.4752594475%, Avg loss: 0.649913
Train Accuracy: 68.4791074763%, Avg loss: 0.649885
Epoch 2
-------------------------------
loss: 0.935367  [    0/ 5994]
loss: 0.926505  [ 1600/ 5994]
loss: 0.915012  [ 3200/ 5994]
loss: 0.897669  [ 4800/ 5994]
Train Accuracy: 82.3855372466%, Avg loss: 0.600732
Train Accuracy: 82.3512165547%, Avg loss: 0.600775
Epoch 3
-------------------------------
loss: 0.883447  [    0/ 5994]
loss: 0.871884  [ 1600/ 5994]
loss: 0.851232  [ 3200/ 5994]
loss: 0.841057  [ 4800/ 5994]
Train Accuracy: 87.9599685155%, Avg loss: 0.550970
Train Accuracy: 87.9382849632%, Avg loss: 0.551170
Epoch 4
-------------------------------
loss: 0.832213  [    0/ 5994]
loss: 0.819149  [ 1600/ 5994]
loss: 0.808159  [ 3200/ 5994]
loss: 0.808283  [ 4800/ 5994]
Train Accuracy: 89.5858465303%, Avg loss: 0.511542
Tra

KeyboardInterrupt: 

```txt
Epoch 1
-------------------------------
loss: 0.995248  [    0/ 5994]
loss: 0.982231  [ 1600/ 5994]
loss: 0.963741  [ 3200/ 5994]
loss: 0.949623  [ 4800/ 5994]
Train Accuracy: 68.4752594475%, Avg loss: 0.649913
Train Accuracy: 68.4791074763%, Avg loss: 0.649885
Epoch 2
-------------------------------
loss: 0.935367  [    0/ 5994]
loss: 0.926505  [ 1600/ 5994]
loss: 0.915012  [ 3200/ 5994]
loss: 0.897669  [ 4800/ 5994]
Train Accuracy: 82.3855372466%, Avg loss: 0.600732
Train Accuracy: 82.3512165547%, Avg loss: 0.600775
Epoch 3
-------------------------------
loss: 0.883447  [    0/ 5994]
loss: 0.871884  [ 1600/ 5994]
loss: 0.851232  [ 3200/ 5994]
loss: 0.841057  [ 4800/ 5994]
Train Accuracy: 87.9599685155%, Avg loss: 0.550970
Train Accuracy: 87.9382849632%, Avg loss: 0.551170
Epoch 4
-------------------------------
loss: 0.832213  [    0/ 5994]
loss: 0.819149  [ 1600/ 5994]
loss: 0.808159  [ 3200/ 5994]
loss: 0.808283  [ 4800/ 5994]
Train Accuracy: 89.5858465303%, Avg loss: 0.511542
Train Accuracy: 89.5476531868%, Avg loss: 0.511791
Epoch 5
-------------------------------
loss: 0.796610  [    0/ 5994]
loss: 0.789368  [ 1600/ 5994]
loss: 0.778142  [ 3200/ 5994]
loss: 0.768972  [ 4800/ 5994]
Train Accuracy: 90.0534615812%, Avg loss: 0.480150
Train Accuracy: 90.0226693396%, Avg loss: 0.480381
Epoch 6
-------------------------------
loss: 0.760993  [    0/ 5994]
loss: 0.744896  [ 1600/ 5994]
loss: 0.741273  [ 3200/ 5994]
loss: 0.730483  [ 4800/ 5994]
Train Accuracy: 90.1621172455%, Avg loss: 0.451749
Train Accuracy: 90.1244545640%, Avg loss: 0.452169
Saved PyTorch Model State to model.pth
Epoch 7
-------------------------------
loss: 0.728736  [    0/ 5994]
loss: 0.713892  [ 1600/ 5994]
loss: 0.716468  [ 3200/ 5994]
loss: 0.707501  [ 4800/ 5994]
Train Accuracy: 90.1968742247%, Avg loss: 0.431276
Train Accuracy: 90.1607985272%, Avg loss: 0.431684
Epoch 8
-------------------------------
loss: 0.705822  [    0/ 5994]
loss: 0.702310  [ 1600/ 5994]
loss: 0.696289  [ 3200/ 5994]
loss: 0.688629  [ 4800/ 5994]
Train Accuracy: 90.2034513146%, Avg loss: 0.414381
Train Accuracy: 90.1715855483%, Avg loss: 0.414706
Epoch 9
-------------------------------
loss: 0.686077  [    0/ 5994]
loss: 0.677549  [ 1600/ 5994]
loss: 0.669427  [ 3200/ 5994]
loss: 0.667801  [ 4800/ 5994]
Train Accuracy: 90.2167124389%, Avg loss: 0.396288
Train Accuracy: 90.1822619332%, Avg loss: 0.396707
Epoch 10
-------------------------------
loss: 0.662504  [    0/ 5994]
loss: 0.654453  [ 1600/ 5994]
loss: 0.655881  [ 3200/ 5994]
loss: 0.646377  [ 4800/ 5994]
Train Accuracy: 90.2293318960%, Avg loss: 0.381797
Train Accuracy: 90.1893426445%, Avg loss: 0.382287
Epoch 11
-------------------------------
loss: 0.637950  [    0/ 5994]
loss: 0.640434  [ 1600/ 5994]
loss: 0.644704  [ 3200/ 5994]
loss: 0.625228  [ 4800/ 5994]
Train Accuracy: 90.2357485691%, Avg loss: 0.370060
Train Accuracy: 90.2021211156%, Avg loss: 0.370559
Saved PyTorch Model State to model.pth
Epoch 12
-------------------------------
loss: 0.635907  [    0/ 5994]
loss: 0.632162  [ 1600/ 5994]
loss: 0.616577  [ 3200/ 5994]
loss: 0.622630  [ 4800/ 5994]
Train Accuracy: 90.2476194143%, Avg loss: 0.360723
Train Accuracy: 90.2094230991%, Avg loss: 0.361210
Epoch 13
-------------------------------
loss: 0.609326  [    0/ 5994]
loss: 0.611071  [ 1600/ 5994]
loss: 0.598987  [ 3200/ 5994]
loss: 0.608664  [ 4800/ 5994]
Train Accuracy: 90.2498117776%, Avg loss: 0.351637
Train Accuracy: 90.2141251339%, Avg loss: 0.352110
Epoch 14
-------------------------------
loss: 0.604138  [    0/ 5994]
loss: 0.594775  [ 1600/ 5994]
loss: 0.601143  [ 3200/ 5994]
loss: 0.588821  [ 4800/ 5994]
Train Accuracy: 90.2592763704%, Avg loss: 0.344142
Train Accuracy: 90.2182739881%, Avg loss: 0.344661
Epoch 15
-------------------------------
loss: 0.607228  [    0/ 5994]
loss: 0.582538  [ 1600/ 5994]
loss: 0.585876  [ 3200/ 5994]
loss: 0.579084  [ 4800/ 5994]
Train Accuracy: 90.2621638733%, Avg loss: 0.336248
Train Accuracy: 90.2202101201%, Avg loss: 0.336821
Epoch 16
-------------------------------
loss: 0.572517  [    0/ 5994]
loss: 0.577649  [ 1600/ 5994]
loss: 0.568022  [ 3200/ 5994]
loss: 0.573063  [ 4800/ 5994]
Train Accuracy: 90.2579930358%, Avg loss: 0.330669
Train Accuracy: 90.2238611118%, Avg loss: 0.331210
Saved PyTorch Model State to model.pth
Epoch 17
-------------------------------
loss: 0.575645  [    0/ 5994]
loss: 0.578575  [ 1600/ 5994]
loss: 0.569510  [ 3200/ 5994]
loss: 0.552840  [ 4800/ 5994]
Train Accuracy: 90.2600784545%, Avg loss: 0.324646
Train Accuracy: 90.2297248259%, Avg loss: 0.325226
Epoch 18
-------------------------------
loss: 0.561463  [    0/ 5994]
loss: 0.560933  [ 1600/ 5994]
loss: 0.558202  [ 3200/ 5994]
loss: 0.544294  [ 4800/ 5994]
Train Accuracy: 90.2598110931%, Avg loss: 0.319770
Train Accuracy: 90.2299460981%, Avg loss: 0.320356
Epoch 19
-------------------------------
loss: 0.537763  [    0/ 5994]
loss: 0.561639  [ 1600/ 5994]
loss: 0.544896  [ 3200/ 5994]
loss: 0.572863  [ 4800/ 5994]
Train Accuracy: 90.2597041486%, Avg loss: 0.315284
Train Accuracy: 90.2248568369%, Avg loss: 0.315866
Epoch 20
-------------------------------
loss: 0.545473  [    0/ 5994]
loss: 0.541890  [ 1600/ 5994]
loss: 0.537333  [ 3200/ 5994]
loss: 0.541058  [ 4800/ 5994]
Train Accuracy: 90.2616826228%, Avg loss: 0.311179
Train Accuracy: 90.2298907800%, Avg loss: 0.311734
Epoch 21
-------------------------------
loss: 0.557100  [    0/ 5994]
loss: 0.546219  [ 1600/ 5994]
loss: 0.518626  [ 3200/ 5994]
loss: 0.517826  [ 4800/ 5994]
Train Accuracy: 90.2613617891%, Avg loss: 0.307902
Train Accuracy: 90.2302226884%, Avg loss: 0.308523
Saved PyTorch Model State to model.pth
Epoch 22
-------------------------------
loss: 0.527747  [    0/ 5994]
loss: 0.530077  [ 1600/ 5994]
loss: 0.525458  [ 3200/ 5994]
loss: 0.530318  [ 4800/ 5994]
Train Accuracy: 90.2623242901%, Avg loss: 0.304370
Train Accuracy: 90.2317162759%, Avg loss: 0.305083
Epoch 23
-------------------------------
loss: 0.527351  [    0/ 5994]
loss: 0.520400  [ 1600/ 5994]
loss: 0.523800  [ 3200/ 5994]
loss: 0.516378  [ 4800/ 5994]
Train Accuracy: 90.2600249822%, Avg loss: 0.301141
Train Accuracy: 90.2302226884%, Avg loss: 0.301806
Epoch 24
-------------------------------
loss: 0.516822  [    0/ 5994]
loss: 0.521515  [ 1600/ 5994]
loss: 0.505041  [ 3200/ 5994]
loss: 0.510363  [ 4800/ 5994]
Train Accuracy: 90.2638215138%, Avg loss: 0.298414
Train Accuracy: 90.2299460981%, Avg loss: 0.299058
Epoch 25
-------------------------------
loss: 0.514323  [    0/ 5994]
loss: 0.514387  [ 1600/ 5994]
loss: 0.517053  [ 3200/ 5994]
loss: 0.517656  [ 4800/ 5994]
Train Accuracy: 90.2668159613%, Avg loss: 0.296354
Train Accuracy: 90.2349800412%, Avg loss: 0.297029
Epoch 26
-------------------------------
loss: 0.506708  [    0/ 5994]
loss: 0.502245  [ 1600/ 5994]
loss: 0.504413  [ 3200/ 5994]
loss: 0.516667  [ 4800/ 5994]
Train Accuracy: 90.2689548523%, Avg loss: 0.294128
Train Accuracy: 90.2299460981%, Avg loss: 0.294802
Saved PyTorch Model State to model.pth
Epoch 27
-------------------------------
loss: 0.530050  [    0/ 5994]
loss: 0.501671  [ 1600/ 5994]
loss: 0.497381  [ 3200/ 5994]
loss: 0.499556  [ 4800/ 5994]
Train Accuracy: 90.2670833226%, Avg loss: 0.291399
Train Accuracy: 90.2332098634%, Avg loss: 0.292122
Epoch 28
-------------------------------
loss: 0.505303  [    0/ 5994]
loss: 0.499008  [ 1600/ 5994]
loss: 0.490650  [ 3200/ 5994]
loss: 0.503356  [ 4800/ 5994]
Train Accuracy: 90.2674041563%, Avg loss: 0.289365
Train Accuracy: 90.2328779551%, Avg loss: 0.290156
Epoch 29
-------------------------------
loss: 0.490573  [    0/ 5994]
loss: 0.492244  [ 1600/ 5994]
loss: 0.509332  [ 3200/ 5994]
loss: 0.495175  [ 4800/ 5994]
Train Accuracy: 90.2677249899%, Avg loss: 0.287495
Train Accuracy: 90.2317715940%, Avg loss: 0.288295
Epoch 30
-------------------------------
loss: 0.491855  [    0/ 5994]
loss: 0.479464  [ 1600/ 5994]
loss: 0.506117  [ 3200/ 5994]
loss: 0.500194  [ 4800/ 5994]
Train Accuracy: 90.2685805464%, Avg loss: 0.286175
Train Accuracy: 90.2355332218%, Avg loss: 0.286982
Epoch 31
-------------------------------
loss: 0.460511  [    0/ 5994]
loss: 0.488646  [ 1600/ 5994]
loss: 0.494746  [ 3200/ 5994]
loss: 0.494605  [ 4800/ 5994]
Train Accuracy: 90.2666555444%, Avg loss: 0.284499
Train Accuracy: 90.2326013648%, Avg loss: 0.285351
Saved PyTorch Model State to model.pth
Epoch 32
-------------------------------
loss: 0.488034  [    0/ 5994]
loss: 0.482015  [ 1600/ 5994]
loss: 0.515846  [ 3200/ 5994]
loss: 0.458671  [ 4800/ 5994]
Train Accuracy: 90.2732861066%, Avg loss: 0.283154
Train Accuracy: 90.2338736801%, Avg loss: 0.283988
Epoch 33
-------------------------------
loss: 0.498128  [    0/ 5994]
loss: 0.468897  [ 1600/ 5994]
loss: 0.485292  [ 3200/ 5994]
loss: 0.473364  [ 4800/ 5994]
Train Accuracy: 90.2717888829%, Avg loss: 0.281907
Train Accuracy: 90.2296695078%, Avg loss: 0.282774
Epoch 34
-------------------------------
loss: 0.485846  [    0/ 5994]
loss: 0.489214  [ 1600/ 5994]
loss: 0.477780  [ 3200/ 5994]
loss: 0.490469  [ 4800/ 5994]
Train Accuracy: 90.2728048561%, Avg loss: 0.280831
Train Accuracy: 90.2332098634%, Avg loss: 0.281658
Epoch 35
-------------------------------
loss: 0.477644  [    0/ 5994]
loss: 0.490373  [ 1600/ 5994]
loss: 0.466896  [ 3200/ 5994]
loss: 0.496364  [ 4800/ 5994]
Train Accuracy: 90.2703986037%, Avg loss: 0.279750
Train Accuracy: 90.2321035023%, Avg loss: 0.280565
Epoch 36
-------------------------------
loss: 0.474170  [    0/ 5994]
loss: 0.475452  [ 1600/ 5994]
loss: 0.487443  [ 3200/ 5994]
loss: 0.472731  [ 4800/ 5994]
Train Accuracy: 90.2687944355%, Avg loss: 0.278697
Train Accuracy: 90.2293375995%, Avg loss: 0.279573
Saved PyTorch Model State to model.pth
Epoch 37
-------------------------------
loss: 0.467757  [    0/ 5994]
loss: 0.467732  [ 1600/ 5994]
loss: 0.471772  [ 3200/ 5994]
loss: 0.487333  [ 4800/ 5994]
Train Accuracy: 90.2695965196%, Avg loss: 0.277806
Train Accuracy: 90.2302780064%, Avg loss: 0.278720
Epoch 38
-------------------------------
loss: 0.465687  [    0/ 5994]
loss: 0.471952  [ 1600/ 5994]
loss: 0.460818  [ 3200/ 5994]
loss: 0.432583  [ 4800/ 5994]
Train Accuracy: 90.2726979116%, Avg loss: 0.276531
Train Accuracy: 90.2301120523%, Avg loss: 0.277406
Epoch 39
-------------------------------
loss: 0.468798  [    0/ 5994]
loss: 0.450745  [ 1600/ 5994]
loss: 0.462258  [ 3200/ 5994]
loss: 0.470842  [ 4800/ 5994]
Train Accuracy: 90.2737138848%, Avg loss: 0.275526
Train Accuracy: 90.2312737314%, Avg loss: 0.276448
Epoch 40
-------------------------------
loss: 0.440788  [    0/ 5994]
loss: 0.475957  [ 1600/ 5994]
loss: 0.452276  [ 3200/ 5994]
loss: 0.433879  [ 4800/ 5994]
Train Accuracy: 90.2695965196%, Avg loss: 0.274602
Train Accuracy: 90.2323800926%, Avg loss: 0.275522
Epoch 41
-------------------------------
loss: 0.463164  [    0/ 5994]
loss: 0.478673  [ 1600/ 5994]
loss: 0.446708  [ 3200/ 5994]
loss: 0.457043  [ 4800/ 5994]
Train Accuracy: 90.2689013800%, Avg loss: 0.274106
Train Accuracy: 90.2323247745%, Avg loss: 0.275014
Saved PyTorch Model State to model.pth
Epoch 42
-------------------------------
loss: 0.453209  [    0/ 5994]
loss: 0.462536  [ 1600/ 5994]
loss: 0.450238  [ 3200/ 5994]
loss: 0.435973  [ 4800/ 5994]
Train Accuracy: 90.2729652730%, Avg loss: 0.273542
Train Accuracy: 90.2335970898%, Avg loss: 0.274473
Epoch 43
-------------------------------
loss: 0.468287  [    0/ 5994]
loss: 0.463966  [ 1600/ 5994]
loss: 0.466669  [ 3200/ 5994]
loss: 0.448576  [ 4800/ 5994]
Train Accuracy: 90.2720027720%, Avg loss: 0.272685
Train Accuracy: 90.2302780064%, Avg loss: 0.273609
Epoch 44
-------------------------------
loss: 0.450267  [    0/ 5994]
loss: 0.459851  [ 1600/ 5994]
loss: 0.461809  [ 3200/ 5994]
loss: 0.443043  [ 4800/ 5994]
Train Accuracy: 90.2746763858%, Avg loss: 0.272206
Train Accuracy: 90.2289503731%, Avg loss: 0.273112
Epoch 45
-------------------------------
loss: 0.457646  [    0/ 5994]
loss: 0.473678  [ 1600/ 5994]
loss: 0.457001  [ 3200/ 5994]
loss: 0.434435  [ 4800/ 5994]
Train Accuracy: 90.2711472156%, Avg loss: 0.271419
Train Accuracy: 90.2316056398%, Avg loss: 0.272286
Epoch 46
-------------------------------
loss: 0.446182  [    0/ 5994]
loss: 0.466875  [ 1600/ 5994]
loss: 0.467152  [ 3200/ 5994]
loss: 0.463438  [ 4800/ 5994]
Train Accuracy: 90.2740881908%, Avg loss: 0.271094
Train Accuracy: 90.2273461494%, Avg loss: 0.272037
Saved PyTorch Model State to model.pth
Epoch 47
-------------------------------
loss: 0.443169  [    0/ 5994]
loss: 0.445948  [ 1600/ 5994]
loss: 0.446772  [ 3200/ 5994]
loss: 0.446944  [ 4800/ 5994]
Train Accuracy: 90.2706659651%, Avg loss: 0.270207
Train Accuracy: 90.2342055885%, Avg loss: 0.271137
Epoch 48
-------------------------------
loss: 0.424590  [    0/ 5994]
loss: 0.454719  [ 1600/ 5994]
loss: 0.443141  [ 3200/ 5994]
loss: 0.447933  [ 4800/ 5994]
Train Accuracy: 90.2709333265%, Avg loss: 0.269848
Train Accuracy: 90.2270142411%, Avg loss: 0.270832
Epoch 49
-------------------------------
loss: 0.458216  [    0/ 5994]
loss: 0.436315  [ 1600/ 5994]
loss: 0.436929  [ 3200/ 5994]
loss: 0.431866  [ 4800/ 5994]
Train Accuracy: 90.2715749938%, Avg loss: 0.269126
Train Accuracy: 90.2273461494%, Avg loss: 0.270063
Epoch 50
-------------------------------
loss: 0.470365  [    0/ 5994]
loss: 0.408197  [ 1600/ 5994]
loss: 0.445021  [ 3200/ 5994]
loss: 0.441090  [ 4800/ 5994]
Train Accuracy: 90.2700777701%, Avg loss: 0.268997
Train Accuracy: 90.2317162759%, Avg loss: 0.269902
Epoch 51
-------------------------------
loss: 0.417361  [    0/ 5994]
loss: 0.430869  [ 1600/ 5994]
loss: 0.459444  [ 3200/ 5994]
loss: 0.450040  [ 4800/ 5994]
Train Accuracy: 90.2725909670%, Avg loss: 0.268441
Train Accuracy: 90.2288950550%, Avg loss: 0.269379
Saved PyTorch Model State to model.pth
Epoch 52
-------------------------------
loss: 0.441169  [    0/ 5994]
loss: 0.419893  [ 1600/ 5994]
loss: 0.457551  [ 3200/ 5994]
loss: 0.447944  [ 4800/ 5994]
Train Accuracy: 90.2730187452%, Avg loss: 0.267951
Train Accuracy: 90.2263504244%, Avg loss: 0.268920
Epoch 53
-------------------------------
loss: 0.440115  [    0/ 5994]
loss: 0.447646  [ 1600/ 5994]
loss: 0.430086  [ 3200/ 5994]
loss: 0.443865  [ 4800/ 5994]
Train Accuracy: 90.2742486076%, Avg loss: 0.267648
Train Accuracy: 90.2308311870%, Avg loss: 0.268635
Epoch 54
-------------------------------
loss: 0.461924  [    0/ 5994]
loss: 0.449335  [ 1600/ 5994]
loss: 0.455032  [ 3200/ 5994]
loss: 0.441744  [ 4800/ 5994]
Train Accuracy: 90.2772430550%, Avg loss: 0.267141
Train Accuracy: 90.2299460981%, Avg loss: 0.268171
Epoch 55
-------------------------------
loss: 0.439017  [    0/ 5994]
loss: 0.450847  [ 1600/ 5994]
loss: 0.453026  [ 3200/ 5994]
loss: 0.444403  [ 4800/ 5994]
Train Accuracy: 90.2715749938%, Avg loss: 0.266867
Train Accuracy: 90.2300014161%, Avg loss: 0.267846
Epoch 56
-------------------------------
loss: 0.412853  [    0/ 5994]
loss: 0.441923  [ 1600/ 5994]
loss: 0.464761  [ 3200/ 5994]
loss: 0.457154  [ 4800/ 5994]
Train Accuracy: 90.2737673571%, Avg loss: 0.266652
Train Accuracy: 90.2307205509%, Avg loss: 0.267633
Saved PyTorch Model State to model.pth
Epoch 57
-------------------------------
loss: 0.430735  [    0/ 5994]
loss: 0.453065  [ 1600/ 5994]
loss: 0.436648  [ 3200/ 5994]
loss: 0.423812  [ 4800/ 5994]
Train Accuracy: 90.2748368026%, Avg loss: 0.266112
Train Accuracy: 90.2316056398%, Avg loss: 0.267109
Epoch 58
-------------------------------
loss: 0.424122  [    0/ 5994]
loss: 0.429667  [ 1600/ 5994]
loss: 0.429878  [ 3200/ 5994]
loss: 0.448172  [ 4800/ 5994]
Train Accuracy: 90.2757993036%, Avg loss: 0.265797
Train Accuracy: 90.2314396856%, Avg loss: 0.266767
Epoch 59
-------------------------------
loss: 0.407004  [    0/ 5994]
loss: 0.398944  [ 1600/ 5994]
loss: 0.427894  [ 3200/ 5994]
loss: 0.418925  [ 4800/ 5994]
Train Accuracy: 90.2754784699%, Avg loss: 0.265532
Train Accuracy: 90.2336524079%, Avg loss: 0.266524
Epoch 60
-------------------------------
loss: 0.421025  [    0/ 5994]
loss: 0.445859  [ 1600/ 5994]
loss: 0.419018  [ 3200/ 5994]
loss: 0.422669  [ 4800/ 5994]
Train Accuracy: 90.2741951353%, Avg loss: 0.265206
Train Accuracy: 90.2313843676%, Avg loss: 0.266199
Epoch 61
-------------------------------
loss: 0.430096  [    0/ 5994]
loss: 0.426142  [ 1600/ 5994]
loss: 0.427479  [ 3200/ 5994]
loss: 0.424541  [ 4800/ 5994]
Train Accuracy: 90.2750506917%, Avg loss: 0.264775
Train Accuracy: 90.2316609578%, Avg loss: 0.265837
Saved PyTorch Model State to model.pth
Epoch 62
-------------------------------
loss: 0.431877  [    0/ 5994]
loss: 0.427654  [ 1600/ 5994]
loss: 0.413098  [ 3200/ 5994]
loss: 0.441999  [ 4800/ 5994]
Train Accuracy: 90.2724305502%, Avg loss: 0.264633
Train Accuracy: 90.2312737314%, Avg loss: 0.265626
Epoch 63
-------------------------------
loss: 0.419311  [    0/ 5994]
loss: 0.426686  [ 1600/ 5994]
loss: 0.425120  [ 3200/ 5994]
loss: 0.432208  [ 4800/ 5994]
Train Accuracy: 90.2728048561%, Avg loss: 0.264289
Train Accuracy: 90.2293929175%, Avg loss: 0.265299
Epoch 64
-------------------------------
loss: 0.424628  [    0/ 5994]
loss: 0.441286  [ 1600/ 5994]
loss: 0.408460  [ 3200/ 5994]
loss: 0.434671  [ 4800/ 5994]
Train Accuracy: 90.2754784699%, Avg loss: 0.264308
Train Accuracy: 90.2295588717%, Avg loss: 0.265348
Epoch 65
-------------------------------
loss: 0.439351  [    0/ 5994]
loss: 0.434470  [ 1600/ 5994]
loss: 0.456708  [ 3200/ 5994]
loss: 0.406885  [ 4800/ 5994]
Train Accuracy: 90.2737673571%, Avg loss: 0.263890
Train Accuracy: 90.2285631467%, Avg loss: 0.264913
Epoch 66
-------------------------------
loss: 0.405011  [    0/ 5994]
loss: 0.418739  [ 1600/ 5994]
loss: 0.434380  [ 3200/ 5994]
loss: 0.414224  [ 4800/ 5994]
Train Accuracy: 90.2747833303%, Avg loss: 0.263693
Train Accuracy: 90.2274567855%, Avg loss: 0.264705
Saved PyTorch Model State to model.pth
Epoch 67
-------------------------------
loss: 0.414586  [    0/ 5994]
loss: 0.429823  [ 1600/ 5994]
loss: 0.407696  [ 3200/ 5994]
loss: 0.423288  [ 4800/ 5994]
Train Accuracy: 90.2744624967%, Avg loss: 0.263454
Train Accuracy: 90.2281206022%, Avg loss: 0.264525
Epoch 68
-------------------------------
loss: 0.435827  [    0/ 5994]
loss: 0.406091  [ 1600/ 5994]
loss: 0.455876  [ 3200/ 5994]
loss: 0.427180  [ 4800/ 5994]
Train Accuracy: 90.2768152768%, Avg loss: 0.263227
Train Accuracy: 90.2309418231%, Avg loss: 0.264280
Epoch 69
-------------------------------
loss: 0.433998  [    0/ 5994]
loss: 0.411176  [ 1600/ 5994]
loss: 0.416877  [ 3200/ 5994]
loss: 0.404826  [ 4800/ 5994]
Train Accuracy: 90.2756388867%, Avg loss: 0.263067
Train Accuracy: 90.2225334785%, Avg loss: 0.264093
Epoch 70
-------------------------------
loss: 0.430936  [    0/ 5994]
loss: 0.425911  [ 1600/ 5994]
loss: 0.420318  [ 3200/ 5994]
loss: 0.433640  [ 4800/ 5994]
Train Accuracy: 90.2777243055%, Avg loss: 0.262815
Train Accuracy: 90.2290056911%, Avg loss: 0.263880
Epoch 71
-------------------------------
loss: 0.424033  [    0/ 5994]
loss: 0.426073  [ 1600/ 5994]
loss: 0.414280  [ 3200/ 5994]
loss: 0.445066  [ 4800/ 5994]
Train Accuracy: 90.2739277739%, Avg loss: 0.262616
Train Accuracy: 90.2319375481%, Avg loss: 0.263665
Saved PyTorch Model State to model.pth
Epoch 72
-------------------------------
loss: 0.427292  [    0/ 5994]
loss: 0.422659  [ 1600/ 5994]
loss: 0.430682  [ 3200/ 5994]
loss: 0.395652  [ 4800/ 5994]
Train Accuracy: 90.2772965273%, Avg loss: 0.262476
Train Accuracy: 90.2309418231%, Avg loss: 0.263524
Epoch 73
-------------------------------
loss: 0.404964  [    0/ 5994]
loss: 0.408274  [ 1600/ 5994]
loss: 0.412180  [ 3200/ 5994]
loss: 0.433611  [ 4800/ 5994]
Train Accuracy: 90.2752645808%, Avg loss: 0.262343
Train Accuracy: 90.2319375481%, Avg loss: 0.263422
Epoch 74
-------------------------------
loss: 0.402514  [    0/ 5994]
loss: 0.423888  [ 1600/ 5994]
loss: 0.398510  [ 3200/ 5994]
loss: 0.403827  [ 4800/ 5994]
Train Accuracy: 90.2784729174%, Avg loss: 0.261978
Train Accuracy: 90.2290056911%, Avg loss: 0.263046
Epoch 75
-------------------------------
loss: 0.379601  [    0/ 5994]
loss: 0.405666  [ 1600/ 5994]
loss: 0.428821  [ 3200/ 5994]
loss: 0.414582  [ 4800/ 5994]
Train Accuracy: 90.2743020799%, Avg loss: 0.261822
Train Accuracy: 90.2305545967%, Avg loss: 0.262878
Epoch 76
-------------------------------
loss: 0.405740  [    0/ 5994]
loss: 0.407168  [ 1600/ 5994]
loss: 0.396699  [ 3200/ 5994]
loss: 0.414040  [ 4800/ 5994]
Train Accuracy: 90.2756388868%, Avg loss: 0.261721
Train Accuracy: 90.2235292035%, Avg loss: 0.262822
Saved PyTorch Model State to model.pth
Epoch 77
-------------------------------
loss: 0.407129  [    0/ 5994]
loss: 0.417301  [ 1600/ 5994]
loss: 0.422891  [ 3200/ 5994]
loss: 0.428995  [ 4800/ 5994]
Train Accuracy: 90.2769756936%, Avg loss: 0.261652
Train Accuracy: 90.2305545967%, Avg loss: 0.262733
Epoch 78
-------------------------------
loss: 0.419026  [    0/ 5994]
loss: 0.400376  [ 1600/ 5994]
loss: 0.396794  [ 3200/ 5994]
loss: 0.401818  [ 4800/ 5994]
Train Accuracy: 90.2787402787%, Avg loss: 0.261545
Train Accuracy: 90.2281759203%, Avg loss: 0.262657
Epoch 79
-------------------------------
loss: 0.398764  [    0/ 5994]
loss: 0.425704  [ 1600/ 5994]
loss: 0.394004  [ 3200/ 5994]
loss: 0.433650  [ 4800/ 5994]
Train Accuracy: 90.2769222214%, Avg loss: 0.261291
Train Accuracy: 90.2250781091%, Avg loss: 0.262403
Epoch 80
-------------------------------
loss: 0.401948  [    0/ 5994]
loss: 0.399745  [ 1600/ 5994]
loss: 0.395536  [ 3200/ 5994]
loss: 0.415388  [ 4800/ 5994]
Train Accuracy: 90.2757458313%, Avg loss: 0.261151
Train Accuracy: 90.2274567855%, Avg loss: 0.262211
Epoch 81
-------------------------------
loss: 0.393420  [    0/ 5994]
loss: 0.413344  [ 1600/ 5994]
loss: 0.416516  [ 3200/ 5994]
loss: 0.430873  [ 4800/ 5994]
Train Accuracy: 90.2748902749%, Avg loss: 0.260998
Train Accuracy: 90.2301120523%, Avg loss: 0.262176
Saved PyTorch Model State to model.pth
Epoch 82
-------------------------------
loss: 0.380321  [    0/ 5994]
loss: 0.410131  [ 1600/ 5994]
loss: 0.409616  [ 3200/ 5994]
loss: 0.439276  [ 4800/ 5994]
Train Accuracy: 90.2782590283%, Avg loss: 0.260870
Train Accuracy: 90.2258525619%, Avg loss: 0.262008
Epoch 83
-------------------------------
loss: 0.409592  [    0/ 5994]
loss: 0.418534  [ 1600/ 5994]
loss: 0.392592  [ 3200/ 5994]
loss: 0.386616  [ 4800/ 5994]
Train Accuracy: 90.2788472233%, Avg loss: 0.261046
Train Accuracy: 90.2301120523%, Avg loss: 0.262214
Epoch 84
-------------------------------
loss: 0.431437  [    0/ 5994]
loss: 0.404383  [ 1600/ 5994]
loss: 0.384451  [ 3200/ 5994]
loss: 0.375285  [ 4800/ 5994]
Train Accuracy: 90.2807722252%, Avg loss: 0.260811
Train Accuracy: 90.2309971412%, Avg loss: 0.261943
Epoch 85
-------------------------------
loss: 0.383123  [    0/ 5994]
loss: 0.417130  [ 1600/ 5994]
loss: 0.369756  [ 3200/ 5994]
loss: 0.392708  [ 4800/ 5994]
Train Accuracy: 90.2777777778%, Avg loss: 0.260595
Train Accuracy: 90.2297801439%, Avg loss: 0.261779
Epoch 86
-------------------------------
loss: 0.398228  [    0/ 5994]
loss: 0.416761  [ 1600/ 5994]
loss: 0.436102  [ 3200/ 5994]
loss: 0.434346  [ 4800/ 5994]
Train Accuracy: 90.2783659728%, Avg loss: 0.260532
Train Accuracy: 90.2306099148%, Avg loss: 0.261737
Saved PyTorch Model State to model.pth
Epoch 87
-------------------------------
loss: 0.393766  [    0/ 5994]
loss: 0.415823  [ 1600/ 5994]
loss: 0.439797  [ 3200/ 5994]
loss: 0.420180  [ 4800/ 5994]
Train Accuracy: 90.2744624967%, Avg loss: 0.260467
Train Accuracy: 90.2288397370%, Avg loss: 0.261663
Epoch 88
-------------------------------
loss: 0.395178  [    0/ 5994]
loss: 0.435859  [ 1600/ 5994]
loss: 0.382520  [ 3200/ 5994]
loss: 0.373177  [ 4800/ 5994]
Train Accuracy: 90.2779916669%, Avg loss: 0.260278
Train Accuracy: 90.2268482869%, Avg loss: 0.261456
Epoch 89
-------------------------------
loss: 0.397228  [    0/ 5994]
loss: 0.442601  [ 1600/ 5994]
loss: 0.394329  [ 3200/ 5994]
loss: 0.404670  [ 4800/ 5994]
Train Accuracy: 90.2780986114%, Avg loss: 0.260068
Train Accuracy: 90.2288397370%, Avg loss: 0.261192
Epoch 90
-------------------------------
loss: 0.412672  [    0/ 5994]
loss: 0.420186  [ 1600/ 5994]
loss: 0.409561  [ 3200/ 5994]
loss: 0.437497  [ 4800/ 5994]
Train Accuracy: 90.2812534757%, Avg loss: 0.260071
Train Accuracy: 90.2276780578%, Avg loss: 0.261218
Epoch 91
-------------------------------
loss: 0.422896  [    0/ 5994]
loss: 0.422117  [ 1600/ 5994]
loss: 0.393688  [ 3200/ 5994]
loss: 0.394682  [ 4800/ 5994]
Train Accuracy: 90.2806652807%, Avg loss: 0.259970
Train Accuracy: 90.2281206022%, Avg loss: 0.261143
Saved PyTorch Model State to model.pth
Epoch 92
-------------------------------
loss: 0.389242  [    0/ 5994]
loss: 0.391088  [ 1600/ 5994]
loss: 0.403502  [ 3200/ 5994]
loss: 0.411712  [ 4800/ 5994]
Train Accuracy: 90.2810930589%, Avg loss: 0.259809
Train Accuracy: 90.2284525106%, Avg loss: 0.260943
Epoch 93
-------------------------------
loss: 0.388332  [    0/ 5994]
loss: 0.386871  [ 1600/ 5994]
loss: 0.390043  [ 3200/ 5994]
loss: 0.404890  [ 4800/ 5994]
Train Accuracy: 90.2809326420%, Avg loss: 0.259797
Train Accuracy: 90.2288397370%, Avg loss: 0.260974
Epoch 94
-------------------------------
loss: 0.428321  [    0/ 5994]
loss: 0.390864  [ 1600/ 5994]
loss: 0.385169  [ 3200/ 5994]
loss: 0.422089  [ 4800/ 5994]
Train Accuracy: 90.2801305579%, Avg loss: 0.259691
Train Accuracy: 90.2278440119%, Avg loss: 0.260926
Epoch 95
-------------------------------
loss: 0.379002  [    0/ 5994]
loss: 0.388352  [ 1600/ 5994]
loss: 0.398606  [ 3200/ 5994]
loss: 0.367696  [ 4800/ 5994]
Train Accuracy: 90.2820020876%, Avg loss: 0.259533
Train Accuracy: 90.2266823327%, Avg loss: 0.260718
Epoch 96
-------------------------------
loss: 0.409469  [    0/ 5994]
loss: 0.425841  [ 1600/ 5994]
loss: 0.415023  [ 3200/ 5994]
loss: 0.384909  [ 4800/ 5994]
Train Accuracy: 90.2797562520%, Avg loss: 0.259418
Train Accuracy: 90.2275121036%, Avg loss: 0.260686
Saved PyTorch Model State to model.pth
Epoch 97
-------------------------------
loss: 0.408639  [    0/ 5994]
loss: 0.385276  [ 1600/ 5994]
loss: 0.385390  [ 3200/ 5994]
loss: 0.402108  [ 4800/ 5994]
Train Accuracy: 90.2821090321%, Avg loss: 0.259292
Train Accuracy: 90.2305545967%, Avg loss: 0.260570
Epoch 98
-------------------------------
loss: 0.394231  [    0/ 5994]
loss: 0.395035  [ 1600/ 5994]
loss: 0.398481  [ 3200/ 5994]
loss: 0.399392  [ 4800/ 5994]
Train Accuracy: 90.2834993113%, Avg loss: 0.259337
Train Accuracy: 90.2255206535%, Avg loss: 0.260569
Epoch 99
-------------------------------
loss: 0.420157  [    0/ 5994]
loss: 0.402585  [ 1600/ 5994]
loss: 0.369375  [ 3200/ 5994]
loss: 0.398289  [ 4800/ 5994]
Train Accuracy: 90.2788472233%, Avg loss: 0.259237
Train Accuracy: 90.2272355133%, Avg loss: 0.260464
Epoch 100
-------------------------------
loss: 0.442724  [    0/ 5994]
loss: 0.409104  [ 1600/ 5994]
loss: 0.378110  [ 3200/ 5994]
loss: 0.400240  [ 4800/ 5994]
Train Accuracy: 90.2755319422%, Avg loss: 0.259171
Train Accuracy: 90.2248568369%, Avg loss: 0.260411
Epoch 101
-------------------------------
loss: 0.389177  [    0/ 5994]
loss: 0.411938  [ 1600/ 5994]
loss: 0.402771  [ 3200/ 5994]
loss: 0.392967  [ 4800/ 5994]
Train Accuracy: 90.2834458390%, Avg loss: 0.259010
Train Accuracy: 90.2261844702%, Avg loss: 0.260312
Saved PyTorch Model State to model.pth
Epoch 102
-------------------------------
loss: 0.364319  [    0/ 5994]
loss: 0.389930  [ 1600/ 5994]
loss: 0.406469  [ 3200/ 5994]
loss: 0.397962  [ 4800/ 5994]
Train Accuracy: 90.2809861143%, Avg loss: 0.258929
Train Accuracy: 90.2288950550%, Avg loss: 0.260219
Epoch 103
-------------------------------
loss: 0.402411  [    0/ 5994]
loss: 0.352918  [ 1600/ 5994]
loss: 0.370853  [ 3200/ 5994]
loss: 0.396410  [ 4800/ 5994]
Train Accuracy: 90.2764944432%, Avg loss: 0.258757
Train Accuracy: 90.2269036050%, Avg loss: 0.260014
Epoch 104
-------------------------------
loss: 0.386033  [    0/ 5994]
loss: 0.385498  [ 1600/ 5994]
loss: 0.417208  [ 3200/ 5994]
loss: 0.401992  [ 4800/ 5994]
Train Accuracy: 90.2788472233%, Avg loss: 0.258788
Train Accuracy: 90.2268482869%, Avg loss: 0.260034
Epoch 105
-------------------------------
loss: 0.408963  [    0/ 5994]
loss: 0.397981  [ 1600/ 5994]
loss: 0.425749  [ 3200/ 5994]
loss: 0.397669  [ 4800/ 5994]
Train Accuracy: 90.2813604202%, Avg loss: 0.258696
Train Accuracy: 90.2288397370%, Avg loss: 0.259994
Epoch 106
-------------------------------
loss: 0.382272  [    0/ 5994]
loss: 0.380867  [ 1600/ 5994]
loss: 0.395313  [ 3200/ 5994]
loss: 0.375008  [ 4800/ 5994]
Train Accuracy: 90.2800236134%, Avg loss: 0.258636
Train Accuracy: 90.2264610605%, Avg loss: 0.259898
Saved PyTorch Model State to model.pth
Epoch 107
-------------------------------
loss: 0.389753  [    0/ 5994]
loss: 0.392312  [ 1600/ 5994]
loss: 0.374244  [ 3200/ 5994]
loss: 0.398241  [ 4800/ 5994]
Train Accuracy: 90.2791145847%, Avg loss: 0.258575
Train Accuracy: 90.2310524592%, Avg loss: 0.259807
Epoch 108
-------------------------------
loss: 0.382934  [    0/ 5994]
loss: 0.373445  [ 1600/ 5994]
loss: 0.383900  [ 3200/ 5994]
loss: 0.415889  [ 4800/ 5994]
Train Accuracy: 90.2806652807%, Avg loss: 0.258639
Train Accuracy: 90.2275121036%, Avg loss: 0.259911
Epoch 109
-------------------------------
loss: 0.379498  [    0/ 5994]
loss: 0.364593  [ 1600/ 5994]
loss: 0.382717  [ 3200/ 5994]
loss: 0.366694  [ 4800/ 5994]
Train Accuracy: 90.2792215292%, Avg loss: 0.258518
Train Accuracy: 90.2265716966%, Avg loss: 0.259819
Epoch 110
-------------------------------
loss: 0.392776  [    0/ 5994]
loss: 0.386193  [ 1600/ 5994]
loss: 0.423492  [ 3200/ 5994]
loss: 0.399281  [ 4800/ 5994]
Train Accuracy: 90.2805583361%, Avg loss: 0.258327
Train Accuracy: 90.2264610605%, Avg loss: 0.259605
Epoch 111
-------------------------------
loss: 0.383596  [    0/ 5994]
loss: 0.396054  [ 1600/ 5994]
loss: 0.399001  [ 3200/ 5994]
loss: 0.397066  [ 4800/ 5994]
Train Accuracy: 90.2774034718%, Avg loss: 0.258346
Train Accuracy: 90.2266270147%, Avg loss: 0.259649
Saved PyTorch Model State to model.pth
Epoch 112
-------------------------------
loss: 0.386777  [    0/ 5994]
loss: 0.408340  [ 1600/ 5994]
loss: 0.398696  [ 3200/ 5994]
loss: 0.377541  [ 4800/ 5994]
Train Accuracy: 90.2764944432%, Avg loss: 0.258345
Train Accuracy: 90.2274567855%, Avg loss: 0.259611
Epoch 113
-------------------------------
loss: 0.398984  [    0/ 5994]
loss: 0.395635  [ 1600/ 5994]
loss: 0.385223  [ 3200/ 5994]
loss: 0.384019  [ 4800/ 5994]
Train Accuracy: 90.2802375025%, Avg loss: 0.258197
Train Accuracy: 90.2296141897%, Avg loss: 0.259488
Epoch 114
-------------------------------
loss: 0.403574  [    0/ 5994]
loss: 0.418453  [ 1600/ 5994]
loss: 0.389717  [ 3200/ 5994]
loss: 0.400281  [ 4800/ 5994]
Train Accuracy: 90.2806652807%, Avg loss: 0.258136
Train Accuracy: 90.2300567342%, Avg loss: 0.259367
Epoch 115
-------------------------------
loss: 0.434327  [    0/ 5994]
loss: 0.395188  [ 1600/ 5994]
loss: 0.399438  [ 3200/ 5994]
loss: 0.404976  [ 4800/ 5994]
Train Accuracy: 90.2792215292%, Avg loss: 0.258147
Train Accuracy: 90.2275674216%, Avg loss: 0.259411
Epoch 116
-------------------------------
loss: 0.387133  [    0/ 5994]
loss: 0.366932  [ 1600/ 5994]
loss: 0.388944  [ 3200/ 5994]
loss: 0.390379  [ 4800/ 5994]
Train Accuracy: 90.2845687568%, Avg loss: 0.257979
Train Accuracy: 90.2272908314%, Avg loss: 0.259331
Saved PyTorch Model State to model.pth
Epoch 117
-------------------------------
loss: 0.398962  [    0/ 5994]
loss: 0.412501  [ 1600/ 5994]
loss: 0.396511  [ 3200/ 5994]
loss: 0.436756  [ 4800/ 5994]
Train Accuracy: 90.2825902826%, Avg loss: 0.257913
Train Accuracy: 90.2243589744%, Avg loss: 0.259246
Epoch 118
-------------------------------
loss: 0.414984  [    0/ 5994]
loss: 0.383925  [ 1600/ 5994]
loss: 0.396618  [ 3200/ 5994]
loss: 0.395923  [ 4800/ 5994]
Train Accuracy: 90.2833923667%, Avg loss: 0.257895
Train Accuracy: 90.2236951577%, Avg loss: 0.259211
Epoch 119
-------------------------------
loss: 0.435327  [    0/ 5994]
loss: 0.397117  [ 1600/ 5994]
loss: 0.391693  [ 3200/ 5994]
loss: 0.389126  [ 4800/ 5994]
Train Accuracy: 90.2793819460%, Avg loss: 0.257857
Train Accuracy: 90.2241930202%, Avg loss: 0.259155
Epoch 120
-------------------------------
loss: 0.381533  [    0/ 5994]
loss: 0.397291  [ 1600/ 5994]
loss: 0.369732  [ 3200/ 5994]
loss: 0.387160  [ 4800/ 5994]
Train Accuracy: 90.2786868065%, Avg loss: 0.257799
Train Accuracy: 90.2238611118%, Avg loss: 0.259094
Epoch 121
-------------------------------
loss: 0.401617  [    0/ 5994]
loss: 0.385786  [ 1600/ 5994]
loss: 0.417449  [ 3200/ 5994]
loss: 0.393988  [ 4800/ 5994]
Train Accuracy: 90.2771895827%, Avg loss: 0.257833
Train Accuracy: 90.2227547507%, Avg loss: 0.259173
Saved PyTorch Model State to model.pth
Epoch 122
-------------------------------
loss: 0.363466  [    0/ 5994]
loss: 0.377856  [ 1600/ 5994]
loss: 0.417039  [ 3200/ 5994]
loss: 0.384010  [ 4800/ 5994]
Train Accuracy: 90.2812000034%, Avg loss: 0.257685
Train Accuracy: 90.2223122063%, Avg loss: 0.259020
Epoch 123
-------------------------------
loss: 0.395184  [    0/ 5994]
loss: 0.384317  [ 1600/ 5994]
loss: 0.364261  [ 3200/ 5994]
loss: 0.412160  [ 4800/ 5994]
Train Accuracy: 90.2810930589%, Avg loss: 0.257740
Train Accuracy: 90.2260738341%, Avg loss: 0.259075
Epoch 124
-------------------------------
loss: 0.384129  [    0/ 5994]
loss: 0.400788  [ 1600/ 5994]
loss: 0.393246  [ 3200/ 5994]
loss: 0.389983  [ 4800/ 5994]
Train Accuracy: 90.2821090321%, Avg loss: 0.257673
Train Accuracy: 90.2239164299%, Avg loss: 0.259024
Epoch 125
-------------------------------
loss: 0.361189  [    0/ 5994]
loss: 0.401143  [ 1600/ 5994]
loss: 0.362805  [ 3200/ 5994]
loss: 0.394806  [ 4800/ 5994]
Train Accuracy: 90.2808791698%, Avg loss: 0.257605
Train Accuracy: 90.2241930202%, Avg loss: 0.258961
Epoch 126
-------------------------------
loss: 0.397014  [    0/ 5994]
loss: 0.387794  [ 1600/ 5994]
loss: 0.420561  [ 3200/ 5994]
loss: 0.366826  [ 4800/ 5994]
Train Accuracy: 90.2823229212%, Avg loss: 0.257489
Train Accuracy: 90.2216483896%, Avg loss: 0.258853
Saved PyTorch Model State to model.pth
Epoch 127
-------------------------------
loss: 0.376102  [    0/ 5994]
loss: 0.379364  [ 1600/ 5994]
loss: 0.410034  [ 3200/ 5994]
loss: 0.373855  [ 4800/ 5994]
Train Accuracy: 90.2817347262%, Avg loss: 0.257428
Train Accuracy: 90.2214271174%, Avg loss: 0.258838
Epoch 128
-------------------------------
loss: 0.415674  [    0/ 5994]
loss: 0.368097  [ 1600/ 5994]
loss: 0.359612  [ 3200/ 5994]
loss: 0.422188  [ 4800/ 5994]
Train Accuracy: 90.2783659728%, Avg loss: 0.257359
Train Accuracy: 90.2275121036%, Avg loss: 0.258794
Epoch 129
-------------------------------
loss: 0.419365  [    0/ 5994]
loss: 0.374879  [ 1600/ 5994]
loss: 0.430512  [ 3200/ 5994]
loss: 0.383423  [ 4800/ 5994]
Train Accuracy: 90.2843548677%, Avg loss: 0.257376
Train Accuracy: 90.2287844189%, Avg loss: 0.258746
Epoch 130
-------------------------------
loss: 0.390524  [    0/ 5994]
loss: 0.392478  [ 1600/ 5994]
loss: 0.368108  [ 3200/ 5994]
loss: 0.371816  [ 4800/ 5994]
Train Accuracy: 90.2828041717%, Avg loss: 0.257304
Train Accuracy: 90.2257972438%, Avg loss: 0.258767
Epoch 131
-------------------------------
loss: 0.364479  [    0/ 5994]
loss: 0.380931  [ 1600/ 5994]
loss: 0.368755  [ 3200/ 5994]
loss: 0.399442  [ 4800/ 5994]
Train Accuracy: 90.2817347262%, Avg loss: 0.257306
Train Accuracy: 90.2260185161%, Avg loss: 0.258709
Saved PyTorch Model State to model.pth
Epoch 132
-------------------------------
loss: 0.381163  [    0/ 5994]
loss: 0.381067  [ 1600/ 5994]
loss: 0.388022  [ 3200/ 5994]
loss: 0.367778  [ 4800/ 5994]
Train Accuracy: 90.2844083400%, Avg loss: 0.257238
Train Accuracy: 90.2270695591%, Avg loss: 0.258672
Epoch 133
-------------------------------
loss: 0.376373  [    0/ 5994]
loss: 0.394606  [ 1600/ 5994]
loss: 0.376902  [ 3200/ 5994]
loss: 0.399333  [ 4800/ 5994]
Train Accuracy: 90.2848895904%, Avg loss: 0.257191
Train Accuracy: 90.2293375995%, Avg loss: 0.258595
Epoch 134
-------------------------------
loss: 0.372332  [    0/ 5994]
loss: 0.358435  [ 1600/ 5994]
loss: 0.388822  [ 3200/ 5994]
loss: 0.393668  [ 4800/ 5994]
Train Accuracy: 90.2817347262%, Avg loss: 0.257066
Train Accuracy: 90.2262951063%, Avg loss: 0.258554
Epoch 135
-------------------------------
loss: 0.375628  [    0/ 5994]
loss: 0.383553  [ 1600/ 5994]
loss: 0.360118  [ 3200/ 5994]
loss: 0.365707  [ 4800/ 5994]
Train Accuracy: 90.2864402864%, Avg loss: 0.257117
Train Accuracy: 90.2211505271%, Avg loss: 0.258556
Epoch 136
-------------------------------
loss: 0.382220  [    0/ 5994]
loss: 0.398169  [ 1600/ 5994]
loss: 0.390564  [ 3200/ 5994]
loss: 0.380602  [ 4800/ 5994]
Train Accuracy: 90.2846757013%, Avg loss: 0.257034
Train Accuracy: 90.2240823841%, Avg loss: 0.258531
Saved PyTorch Model State to model.pth
Epoch 137
-------------------------------
loss: 0.375265  [    0/ 5994]
loss: 0.375929  [ 1600/ 5994]
loss: 0.386772  [ 3200/ 5994]
loss: 0.379307  [ 4800/ 5994]
Train Accuracy: 90.2861729251%, Avg loss: 0.257043
Train Accuracy: 90.2257419258%, Avg loss: 0.258520
Epoch 138
-------------------------------
loss: 0.391738  [    0/ 5994]
loss: 0.374070  [ 1600/ 5994]
loss: 0.379933  [ 3200/ 5994]
loss: 0.393417  [ 4800/ 5994]
Train Accuracy: 90.2831250053%, Avg loss: 0.257015
Train Accuracy: 90.2312737314%, Avg loss: 0.258499
Epoch 139
-------------------------------
loss: 0.377944  [    0/ 5994]
loss: 0.366418  [ 1600/ 5994]
loss: 0.392308  [ 3200/ 5994]
loss: 0.372508  [ 4800/ 5994]
Train Accuracy: 90.2834993113%, Avg loss: 0.257003
Train Accuracy: 90.2278440119%, Avg loss: 0.258471
Epoch 140
-------------------------------
loss: 0.359804  [    0/ 5994]
loss: 0.408226  [ 1600/ 5994]
loss: 0.399548  [ 3200/ 5994]
loss: 0.373336  [ 4800/ 5994]
Train Accuracy: 90.2819486153%, Avg loss: 0.256869
Train Accuracy: 90.2259078799%, Avg loss: 0.258383
Epoch 141
-------------------------------
loss: 0.368604  [    0/ 5994]
loss: 0.386853  [ 1600/ 5994]
loss: 0.359179  [ 3200/ 5994]
loss: 0.363787  [ 4800/ 5994]
Train Accuracy: 90.2851034795%, Avg loss: 0.256872
Train Accuracy: 90.2254100174%, Avg loss: 0.258399
Saved PyTorch Model State to model.pth
Epoch 142
-------------------------------
loss: 0.381133  [    0/ 5994]
loss: 0.340851  [ 1600/ 5994]
loss: 0.339747  [ 3200/ 5994]
loss: 0.361116  [ 4800/ 5994]
Train Accuracy: 90.2865472310%, Avg loss: 0.256848
Train Accuracy: 90.2235292035%, Avg loss: 0.258430
Epoch 143
-------------------------------
loss: 0.388063  [    0/ 5994]
loss: 0.420799  [ 1600/ 5994]
loss: 0.380583  [ 3200/ 5994]
loss: 0.393558  [ 4800/ 5994]
Train Accuracy: 90.2861194528%, Avg loss: 0.256782
Train Accuracy: 90.2278993300%, Avg loss: 0.258383
Epoch 144
-------------------------------
loss: 0.386562  [    0/ 5994]
loss: 0.358945  [ 1600/ 5994]
loss: 0.374117  [ 3200/ 5994]
loss: 0.368842  [ 4800/ 5994]
Train Accuracy: 90.2858520914%, Avg loss: 0.256855
Train Accuracy: 90.2250227910%, Avg loss: 0.258428
Epoch 145
-------------------------------
loss: 0.381003  [    0/ 5994]
loss: 0.416221  [ 1600/ 5994]
loss: 0.417202  [ 3200/ 5994]
loss: 0.398731  [ 4800/ 5994]
Train Accuracy: 90.2871354260%, Avg loss: 0.256768
Train Accuracy: 90.2230866591%, Avg loss: 0.258360
Epoch 146
-------------------------------
loss: 0.365133  [    0/ 5994]
loss: 0.406008  [ 1600/ 5994]
loss: 0.373567  [ 3200/ 5994]
loss: 0.392648  [ 4800/ 5994]
Train Accuracy: 90.2896486230%, Avg loss: 0.256735
Train Accuracy: 90.2259631980%, Avg loss: 0.258382
Saved PyTorch Model State to model.pth
Epoch 147
-------------------------------
loss: 0.444346  [    0/ 5994]
loss: 0.391910  [ 1600/ 5994]
loss: 0.414946  [ 3200/ 5994]
loss: 0.403343  [ 4800/ 5994]
Train Accuracy: 90.2853708409%, Avg loss: 0.256760
Train Accuracy: 90.2261291522%, Avg loss: 0.258366
Epoch 148
-------------------------------
loss: 0.373537  [    0/ 5994]
loss: 0.397227  [ 1600/ 5994]
loss: 0.376690  [ 3200/ 5994]
loss: 0.367784  [ 4800/ 5994]
Train Accuracy: 90.2830715331%, Avg loss: 0.256780
Train Accuracy: 90.2217590257%, Avg loss: 0.258375
Epoch 149
-------------------------------
loss: 0.368901  [    0/ 5994]
loss: 0.365631  [ 1600/ 5994]
loss: 0.409543  [ 3200/ 5994]
loss: 0.381763  [ 4800/ 5994]
Train Accuracy: 90.2867611201%, Avg loss: 0.256586
Train Accuracy: 90.2217590257%, Avg loss: 0.258194
Epoch 150
-------------------------------
loss: 0.378075  [    0/ 5994]
loss: 0.376585  [ 1600/ 5994]
loss: 0.391383  [ 3200/ 5994]
loss: 0.378248  [ 4800/ 5994]
Train Accuracy: 90.2882583438%, Avg loss: 0.256635
Train Accuracy: 90.2204867104%, Avg loss: 0.258257
Epoch 151
-------------------------------
loss: 0.374559  [    0/ 5994]
loss: 0.365636  [ 1600/ 5994]
loss: 0.375455  [ 3200/ 5994]
loss: 0.398556  [ 4800/ 5994]
Train Accuracy: 90.2830715331%, Avg loss: 0.256589
Train Accuracy: 90.2217590257%, Avg loss: 0.258185
Saved PyTorch Model State to model.pth
Epoch 152
-------------------------------
loss: 0.358252  [    0/ 5994]
loss: 0.360325  [ 1600/ 5994]
loss: 0.358542  [ 3200/ 5994]
loss: 0.401637  [ 4800/ 5994]
Train Accuracy: 90.2828041717%, Avg loss: 0.256544
Train Accuracy: 90.2225334785%, Avg loss: 0.258151
Epoch 153
-------------------------------
loss: 0.374596  [    0/ 5994]
loss: 0.380171  [ 1600/ 5994]
loss: 0.361791  [ 3200/ 5994]
loss: 0.368990  [ 4800/ 5994]
Train Accuracy: 90.2839270895%, Avg loss: 0.256621
Train Accuracy: 90.2197675757%, Avg loss: 0.258249
Epoch 154
-------------------------------
loss: 0.359484  [    0/ 5994]
loss: 0.366535  [ 1600/ 5994]
loss: 0.358307  [ 3200/ 5994]
loss: 0.349556  [ 4800/ 5994]
Train Accuracy: 90.2851569518%, Avg loss: 0.256516
Train Accuracy: 90.2180527159%, Avg loss: 0.258119
Epoch 155
-------------------------------
loss: 0.369490  [    0/ 5994]
loss: 0.397876  [ 1600/ 5994]
loss: 0.369966  [ 3200/ 5994]
loss: 0.372759  [ 4800/ 5994]
Train Accuracy: 90.2811465311%, Avg loss: 0.256449
Train Accuracy: 90.2188824867%, Avg loss: 0.258072
Epoch 156
-------------------------------
loss: 0.363822  [    0/ 5994]
loss: 0.388593  [ 1600/ 5994]
loss: 0.401537  [ 3200/ 5994]
loss: 0.359503  [ 4800/ 5994]
Train Accuracy: 90.2879375102%, Avg loss: 0.256382
Train Accuracy: 90.2220909340%, Avg loss: 0.258015
Saved PyTorch Model State to model.pth
Epoch 157
-------------------------------
loss: 0.345773  [    0/ 5994]
loss: 0.397622  [ 1600/ 5994]
loss: 0.363263  [ 3200/ 5994]
loss: 0.354212  [ 4800/ 5994]
Train Accuracy: 90.2864937587%, Avg loss: 0.256388
Train Accuracy: 90.2158399936%, Avg loss: 0.258016
Epoch 158
-------------------------------
loss: 0.353789  [    0/ 5994]
loss: 0.357376  [ 1600/ 5994]
loss: 0.390586  [ 3200/ 5994]
loss: 0.366605  [ 4800/ 5994]
Train Accuracy: 90.2832319499%, Avg loss: 0.256418
Train Accuracy: 90.2243036563%, Avg loss: 0.258033
Epoch 159
-------------------------------
loss: 0.388266  [    0/ 5994]
loss: 0.377830  [ 1600/ 5994]
loss: 0.382759  [ 3200/ 5994]
loss: 0.360163  [ 4800/ 5994]
Train Accuracy: 90.2821090321%, Avg loss: 0.256342
Train Accuracy: 90.2238611118%, Avg loss: 0.258009
Epoch 160
-------------------------------
loss: 0.362553  [    0/ 5994]
loss: 0.358567  [ 1600/ 5994]
loss: 0.368011  [ 3200/ 5994]
loss: 0.392050  [ 4800/ 5994]
Train Accuracy: 90.2831784776%, Avg loss: 0.256297
Train Accuracy: 90.2224781604%, Avg loss: 0.257927
Epoch 161
-------------------------------
loss: 0.366984  [    0/ 5994]
loss: 0.365100  [ 1600/ 5994]
loss: 0.412863  [ 3200/ 5994]
loss: 0.375698  [ 4800/ 5994]
Train Accuracy: 90.2809861143%, Avg loss: 0.256334
Train Accuracy: 90.2208186187%, Avg loss: 0.257951
Saved PyTorch Model State to model.pth
Epoch 162
-------------------------------
loss: 0.406246  [    0/ 5994]
loss: 0.401485  [ 1600/ 5994]
loss: 0.391922  [ 3200/ 5994]
loss: 0.373166  [ 4800/ 5994]
Train Accuracy: 90.2851034795%, Avg loss: 0.256195
Train Accuracy: 90.2218696618%, Avg loss: 0.257828
Epoch 163
-------------------------------
loss: 0.366812  [    0/ 5994]
loss: 0.365653  [ 1600/ 5994]
loss: 0.343167  [ 3200/ 5994]
loss: 0.379434  [ 4800/ 5994]
Train Accuracy: 90.2865472310%, Avg loss: 0.256297
Train Accuracy: 90.2224228424%, Avg loss: 0.257901
Epoch 164
-------------------------------
loss: 0.386864  [    0/ 5994]
loss: 0.369896  [ 1600/ 5994]
loss: 0.355795  [ 3200/ 5994]
loss: 0.344479  [ 4800/ 5994]
Train Accuracy: 90.2857986191%, Avg loss: 0.256219
Train Accuracy: 90.2170016728%, Avg loss: 0.257870
Epoch 165
-------------------------------
loss: 0.358288  [    0/ 5994]
loss: 0.372246  [ 1600/ 5994]
loss: 0.373856  [ 3200/ 5994]
loss: 0.362046  [ 4800/ 5994]
Train Accuracy: 90.2849430627%, Avg loss: 0.256168
Train Accuracy: 90.2187165326%, Avg loss: 0.257839
Epoch 166
-------------------------------
loss: 0.401601  [    0/ 5994]
loss: 0.363993  [ 1600/ 5994]
loss: 0.367461  [ 3200/ 5994]
loss: 0.360106  [ 4800/ 5994]
Train Accuracy: 90.2842479231%, Avg loss: 0.256122
Train Accuracy: 90.2222568882%, Avg loss: 0.257824
Saved PyTorch Model State to model.pth
Epoch 167
-------------------------------
loss: 0.356458  [    0/ 5994]
loss: 0.375740  [ 1600/ 5994]
loss: 0.351647  [ 3200/ 5994]
loss: 0.362069  [ 4800/ 5994]
Train Accuracy: 90.2815208371%, Avg loss: 0.256153
Train Accuracy: 90.2198228937%, Avg loss: 0.257829
Epoch 168
-------------------------------
loss: 0.399015  [    0/ 5994]
loss: 0.416390  [ 1600/ 5994]
loss: 0.359180  [ 3200/ 5994]
loss: 0.418346  [ 4800/ 5994]
Train Accuracy: 90.2863333419%, Avg loss: 0.256091
Train Accuracy: 90.2189378048%, Avg loss: 0.257779
Epoch 169
-------------------------------
loss: 0.368064  [    0/ 5994]
loss: 0.375349  [ 1600/ 5994]
loss: 0.380240  [ 3200/ 5994]
loss: 0.369186  [ 4800/ 5994]
Train Accuracy: 90.2857986191%, Avg loss: 0.256186
Train Accuracy: 90.2200441659%, Avg loss: 0.257842
Epoch 170
-------------------------------
loss: 0.354360  [    0/ 5994]
loss: 0.403098  [ 1600/ 5994]
loss: 0.353957  [ 3200/ 5994]
loss: 0.361037  [ 4800/ 5994]
Train Accuracy: 90.2859590360%, Avg loss: 0.256088
Train Accuracy: 90.2200994840%, Avg loss: 0.257779
Epoch 171
-------------------------------
loss: 0.411960  [    0/ 5994]
loss: 0.404210  [ 1600/ 5994]
loss: 0.383633  [ 3200/ 5994]
loss: 0.385118  [ 4800/ 5994]
Train Accuracy: 90.2839270895%, Avg loss: 0.256016
Train Accuracy: 90.2170569909%, Avg loss: 0.257757
Saved PyTorch Model State to model.pth
Epoch 172
-------------------------------
loss: 0.376086  [    0/ 5994]
loss: 0.342393  [ 1600/ 5994]
loss: 0.402521  [ 3200/ 5994]
loss: 0.372927  [ 4800/ 5994]
Train Accuracy: 90.2855312578%, Avg loss: 0.255980
Train Accuracy: 90.2172782631%, Avg loss: 0.257716
Epoch 173
-------------------------------
loss: 0.369483  [    0/ 5994]
loss: 0.394909  [ 1600/ 5994]
loss: 0.363217  [ 3200/ 5994]
loss: 0.362678  [ 4800/ 5994]
Train Accuracy: 90.2837132004%, Avg loss: 0.255948
Train Accuracy: 90.2194909854%, Avg loss: 0.257666
Epoch 174
-------------------------------
loss: 0.358710  [    0/ 5994]
loss: 0.379796  [ 1600/ 5994]
loss: 0.372917  [ 3200/ 5994]
loss: 0.370745  [ 4800/ 5994]
Train Accuracy: 90.2859590360%, Avg loss: 0.256010
Train Accuracy: 90.2145123603%, Avg loss: 0.257708
Epoch 175
-------------------------------
loss: 0.382442  [    0/ 5994]
loss: 0.342695  [ 1600/ 5994]
loss: 0.392815  [ 3200/ 5994]
loss: 0.366046  [ 4800/ 5994]
Train Accuracy: 90.2869215369%, Avg loss: 0.255938
Train Accuracy: 90.2178867617%, Avg loss: 0.257683
Epoch 176
-------------------------------
loss: 0.398037  [    0/ 5994]
loss: 0.395679  [ 1600/ 5994]
loss: 0.371313  [ 3200/ 5994]
loss: 0.413333  [ 4800/ 5994]
Train Accuracy: 90.2875632042%, Avg loss: 0.255992
Train Accuracy: 90.2167804006%, Avg loss: 0.257677
Saved PyTorch Model State to model.pth
Epoch 177
-------------------------------
loss: 0.438729  [    0/ 5994]
loss: 0.376206  [ 1600/ 5994]
loss: 0.360985  [ 3200/ 5994]
loss: 0.383184  [ 4800/ 5994]
Train Accuracy: 90.2861729251%, Avg loss: 0.255962
Train Accuracy: 90.2131847269%, Avg loss: 0.257705
Epoch 178
-------------------------------
loss: 0.381035  [    0/ 5994]
loss: 0.370997  [ 1600/ 5994]
loss: 0.359958  [ 3200/ 5994]
loss: 0.369096  [ 4800/ 5994]
Train Accuracy: 90.2875097320%, Avg loss: 0.255905
Train Accuracy: 90.2146783144%, Avg loss: 0.257676
Epoch 179
-------------------------------
loss: 0.411518  [    0/ 5994]
loss: 0.371757  [ 1600/ 5994]
loss: 0.383513  [ 3200/ 5994]
loss: 0.354935  [ 4800/ 5994]
Train Accuracy: 90.2868145924%, Avg loss: 0.255891
Train Accuracy: 90.2161719020%, Avg loss: 0.257627
Epoch 180
-------------------------------
loss: 0.404123  [    0/ 5994]
loss: 0.376022  [ 1600/ 5994]
loss: 0.360175  [ 3200/ 5994]
loss: 0.392984  [ 4800/ 5994]
Train Accuracy: 90.2868680646%, Avg loss: 0.255867
Train Accuracy: 90.2181633520%, Avg loss: 0.257575
Epoch 181
-------------------------------
loss: 0.380396  [    0/ 5994]
loss: 0.387337  [ 1600/ 5994]
loss: 0.388916  [ 3200/ 5994]
loss: 0.382995  [ 4800/ 5994]
Train Accuracy: 90.2883118161%, Avg loss: 0.255821
Train Accuracy: 90.2173888992%, Avg loss: 0.257550
Saved PyTorch Model State to model.pth
Epoch 182
-------------------------------
loss: 0.377225  [    0/ 5994]
loss: 0.375019  [ 1600/ 5994]
loss: 0.390951  [ 3200/ 5994]
loss: 0.364360  [ 4800/ 5994]
Train Accuracy: 90.2867611201%, Avg loss: 0.255836
Train Accuracy: 90.2163378561%, Avg loss: 0.257566
Epoch 183
-------------------------------
loss: 0.381378  [    0/ 5994]
loss: 0.339017  [ 1600/ 5994]
loss: 0.375915  [ 3200/ 5994]
loss: 0.381186  [ 4800/ 5994]
Train Accuracy: 90.2801840302%, Avg loss: 0.255826
Train Accuracy: 90.2168910367%, Avg loss: 0.257596
Epoch 184
-------------------------------
loss: 0.377416  [    0/ 5994]
loss: 0.375616  [ 1600/ 5994]
loss: 0.370867  [ 3200/ 5994]
loss: 0.380358  [ 4800/ 5994]
Train Accuracy: 90.2872423706%, Avg loss: 0.255826
Train Accuracy: 90.2180527159%, Avg loss: 0.257576
Epoch 185
-------------------------------
loss: 0.341446  [    0/ 5994]
loss: 0.428860  [ 1600/ 5994]
loss: 0.348464  [ 3200/ 5994]
loss: 0.359630  [ 4800/ 5994]
Train Accuracy: 90.2897555675%, Avg loss: 0.255812
Train Accuracy: 90.2181080340%, Avg loss: 0.257557
Epoch 186
-------------------------------
loss: 0.370050  [    0/ 5994]
loss: 0.383131  [ 1600/ 5994]
loss: 0.357797  [ 3200/ 5994]
loss: 0.363029  [ 4800/ 5994]
Train Accuracy: 90.2847291736%, Avg loss: 0.255785
Train Accuracy: 90.2183293062%, Avg loss: 0.257544
Saved PyTorch Model State to model.pth
Epoch 187
-------------------------------
loss: 0.368221  [    0/ 5994]
loss: 0.390279  [ 1600/ 5994]
loss: 0.351273  [ 3200/ 5994]
loss: 0.389677  [ 4800/ 5994]
Train Accuracy: 90.2855847300%, Avg loss: 0.255719
Train Accuracy: 90.2162272200%, Avg loss: 0.257486
Epoch 188
-------------------------------
loss: 0.355462  [    0/ 5994]
loss: 0.400251  [ 1600/ 5994]
loss: 0.363607  [ 3200/ 5994]
loss: 0.375004  [ 4800/ 5994]
Train Accuracy: 90.2845687568%, Avg loss: 0.255716
Train Accuracy: 90.2180527159%, Avg loss: 0.257505
Epoch 189
-------------------------------
loss: 0.373183  [    0/ 5994]
loss: 0.372273  [ 1600/ 5994]
loss: 0.347473  [ 3200/ 5994]
loss: 0.400221  [ 4800/ 5994]
Train Accuracy: 90.2846222291%, Avg loss: 0.255729
Train Accuracy: 90.2149549047%, Avg loss: 0.257497
Epoch 190
-------------------------------
loss: 0.404945  [    0/ 5994]
loss: 0.406825  [ 1600/ 5994]
loss: 0.375174  [ 3200/ 5994]
loss: 0.368440  [ 4800/ 5994]
Train Accuracy: 90.2852104241%, Avg loss: 0.255710
Train Accuracy: 90.2145676783%, Avg loss: 0.257495
Epoch 191
-------------------------------
loss: 0.379336  [    0/ 5994]
loss: 0.357334  [ 1600/ 5994]
loss: 0.374210  [ 3200/ 5994]
loss: 0.347009  [ 4800/ 5994]
Train Accuracy: 90.2849965350%, Avg loss: 0.255672
Train Accuracy: 90.2135166352%, Avg loss: 0.257480
Saved PyTorch Model State to model.pth
Epoch 192
-------------------------------
loss: 0.380038  [    0/ 5994]
loss: 0.376758  [ 1600/ 5994]
loss: 0.374956  [ 3200/ 5994]
loss: 0.357995  [ 4800/ 5994]
Train Accuracy: 90.2876701488%, Avg loss: 0.255649
Train Accuracy: 90.2148442686%, Avg loss: 0.257509
Epoch 193
-------------------------------
loss: 0.342530  [    0/ 5994]
loss: 0.366237  [ 1600/ 5994]
loss: 0.339103  [ 3200/ 5994]
loss: 0.340568  [ 4800/ 5994]
Train Accuracy: 90.2881513993%, Avg loss: 0.255607
Train Accuracy: 90.2126868644%, Avg loss: 0.257446
Epoch 194
-------------------------------
loss: 0.368680  [    0/ 5994]
loss: 0.406077  [ 1600/ 5994]
loss: 0.352708  [ 3200/ 5994]
loss: 0.392858  [ 4800/ 5994]
Train Accuracy: 90.2848895904%, Avg loss: 0.255620
Train Accuracy: 90.2149549047%, Avg loss: 0.257488
Epoch 195
-------------------------------
loss: 0.382772  [    0/ 5994]
loss: 0.376781  [ 1600/ 5994]
loss: 0.373646  [ 3200/ 5994]
loss: 0.393802  [ 4800/ 5994]
Train Accuracy: 90.2868145924%, Avg loss: 0.255590
Train Accuracy: 90.2177761256%, Avg loss: 0.257430
Epoch 196
-------------------------------
loss: 0.365587  [    0/ 5994]
loss: 0.379070  [ 1600/ 5994]
loss: 0.338983  [ 3200/ 5994]
loss: 0.377010  [ 4800/ 5994]
Train Accuracy: 90.2857986191%, Avg loss: 0.255590
Train Accuracy: 90.2139038616%, Avg loss: 0.257414
Saved PyTorch Model State to model.pth
Epoch 197
-------------------------------
loss: 0.383828  [    0/ 5994]
loss: 0.368758  [ 1600/ 5994]
loss: 0.386626  [ 3200/ 5994]
loss: 0.347121  [ 4800/ 5994]
Train Accuracy: 90.2862263973%, Avg loss: 0.255550
Train Accuracy: 90.2158399936%, Avg loss: 0.257392
Epoch 198
-------------------------------
loss: 0.391020  [    0/ 5994]
loss: 0.379284  [ 1600/ 5994]
loss: 0.382062  [ 3200/ 5994]
loss: 0.357536  [ 4800/ 5994]
Train Accuracy: 90.2886861220%, Avg loss: 0.255551
Train Accuracy: 90.2142910880%, Avg loss: 0.257412
Epoch 199
-------------------------------
loss: 0.349891  [    0/ 5994]
loss: 0.336228  [ 1600/ 5994]
loss: 0.393261  [ 3200/ 5994]
loss: 0.352846  [ 4800/ 5994]
Train Accuracy: 90.2844618122%, Avg loss: 0.255566
Train Accuracy: 90.2156187214%, Avg loss: 0.257447
Epoch 200
-------------------------------
loss: 0.376280  [    0/ 5994]
loss: 0.367377  [ 1600/ 5994]
loss: 0.372005  [ 3200/ 5994]
loss: 0.376856  [ 4800/ 5994]
Train Accuracy: 90.2841409786%, Avg loss: 0.255520
Train Accuracy: 90.2147336325%, Avg loss: 0.257430
Epoch 201
-------------------------------
loss: 0.368810  [    0/ 5994]
loss: 0.407282  [ 1600/ 5994]
loss: 0.384648  [ 3200/ 5994]
loss: 0.404254  [ 4800/ 5994]
Train Accuracy: 90.2892208448%, Avg loss: 0.255473
Train Accuracy: 90.2193250312%, Avg loss: 0.257395
Saved PyTorch Model State to model.pth
Epoch 202
-------------------------------
loss: 0.357641  [    0/ 5994]
loss: 0.414066  [ 1600/ 5994]
loss: 0.376693  [ 3200/ 5994]
loss: 0.361610  [ 4800/ 5994]
Train Accuracy: 90.2839805618%, Avg loss: 0.255550
Train Accuracy: 90.2157293575%, Avg loss: 0.257392
Epoch 203
-------------------------------
loss: 0.360314  [    0/ 5994]
loss: 0.376714  [ 1600/ 5994]
loss: 0.363743  [ 3200/ 5994]
loss: 0.365849  [ 4800/ 5994]
Train Accuracy: 90.2852104241%, Avg loss: 0.255497
Train Accuracy: 90.2202654382%, Avg loss: 0.257407
Epoch 204
-------------------------------
loss: 0.379836  [    0/ 5994]
loss: 0.346785  [ 1600/ 5994]
loss: 0.351729  [ 3200/ 5994]
loss: 0.378215  [ 4800/ 5994]
Train Accuracy: 90.2824833380%, Avg loss: 0.255497
Train Accuracy: 90.2138485436%, Avg loss: 0.257398
Epoch 205
-------------------------------
loss: 0.361529  [    0/ 5994]
loss: 0.386881  [ 1600/ 5994]
loss: 0.373248  [ 3200/ 5994]
loss: 0.371166  [ 4800/ 5994]
Train Accuracy: 90.2877236211%, Avg loss: 0.255484
Train Accuracy: 90.2172229450%, Avg loss: 0.257366
Epoch 206
-------------------------------
loss: 0.393671  [    0/ 5994]
loss: 0.408270  [ 1600/ 5994]
loss: 0.441988  [ 3200/ 5994]
loss: 0.367486  [ 4800/ 5994]
Train Accuracy: 90.2859590360%, Avg loss: 0.255418
Train Accuracy: 90.2163378561%, Avg loss: 0.257363
Saved PyTorch Model State to model.pth
Epoch 207
-------------------------------
loss: 0.376781  [    0/ 5994]
loss: 0.365946  [ 1600/ 5994]
loss: 0.368015  [ 3200/ 5994]
loss: 0.361921  [ 4800/ 5994]
Train Accuracy: 90.2854777855%, Avg loss: 0.255385
Train Accuracy: 90.2165591284%, Avg loss: 0.257371
Epoch 208
-------------------------------
loss: 0.430184  [    0/ 5994]
loss: 0.370885  [ 1600/ 5994]
loss: 0.374845  [ 3200/ 5994]
loss: 0.385234  [ 4800/ 5994]
Train Accuracy: 90.2856916746%, Avg loss: 0.255391
Train Accuracy: 90.2134613172%, Avg loss: 0.257334
Epoch 209
-------------------------------
loss: 0.388089  [    0/ 5994]
loss: 0.339352  [ 1600/ 5994]
loss: 0.403599  [ 3200/ 5994]
loss: 0.377140  [ 4800/ 5994]
Train Accuracy: 90.2855312578%, Avg loss: 0.255411
Train Accuracy: 90.2169463548%, Avg loss: 0.257353
Epoch 210
-------------------------------
loss: 0.370944  [    0/ 5994]
loss: 0.366954  [ 1600/ 5994]
loss: 0.384662  [ 3200/ 5994]
loss: 0.408471  [ 4800/ 5994]
Train Accuracy: 90.2858520914%, Avg loss: 0.255412
Train Accuracy: 90.2147336325%, Avg loss: 0.257358
Epoch 211
-------------------------------
loss: 0.395020  [    0/ 5994]
loss: 0.374348  [ 1600/ 5994]
loss: 0.365575  [ 3200/ 5994]
loss: 0.360495  [ 4800/ 5994]
Train Accuracy: 90.2825368103%, Avg loss: 0.255330
Train Accuracy: 90.2144017242%, Avg loss: 0.257340
Saved PyTorch Model State to model.pth
Epoch 212
-------------------------------
loss: 0.379526  [    0/ 5994]
loss: 0.362029  [ 1600/ 5994]
loss: 0.375476  [ 3200/ 5994]
loss: 0.340667  [ 4800/ 5994]
Train Accuracy: 90.2885791775%, Avg loss: 0.255348
Train Accuracy: 90.2150102228%, Avg loss: 0.257262
Epoch 213
-------------------------------
loss: 0.371634  [    0/ 5994]
loss: 0.397996  [ 1600/ 5994]
loss: 0.386599  [ 3200/ 5994]
loss: 0.333046  [ 4800/ 5994]
Train Accuracy: 90.2857986191%, Avg loss: 0.255315
Train Accuracy: 90.2148995867%, Avg loss: 0.257267
Epoch 214
-------------------------------
loss: 0.358740  [    0/ 5994]
loss: 0.370591  [ 1600/ 5994]
loss: 0.363830  [ 3200/ 5994]
loss: 0.415066  [ 4800/ 5994]
Train Accuracy: 90.2869750092%, Avg loss: 0.255307
Train Accuracy: 90.2126315463%, Avg loss: 0.257291
Epoch 215
-------------------------------
loss: 0.395196  [    0/ 5994]
loss: 0.394941  [ 1600/ 5994]
loss: 0.384904  [ 3200/ 5994]
loss: 0.335197  [ 4800/ 5994]
Train Accuracy: 90.2891139002%, Avg loss: 0.255260
Train Accuracy: 90.2174442173%, Avg loss: 0.257259
Epoch 216
-------------------------------
loss: 0.362164  [    0/ 5994]
loss: 0.362120  [ 1600/ 5994]
loss: 0.362351  [ 3200/ 5994]
loss: 0.419172  [ 4800/ 5994]
Train Accuracy: 90.2860659805%, Avg loss: 0.255318
Train Accuracy: 90.2150102228%, Avg loss: 0.257296
Saved PyTorch Model State to model.pth
Epoch 217
-------------------------------
loss: 0.366030  [    0/ 5994]
loss: 0.373659  [ 1600/ 5994]
loss: 0.354551  [ 3200/ 5994]
loss: 0.348682  [ 4800/ 5994]
Train Accuracy: 90.2853173687%, Avg loss: 0.255424
Train Accuracy: 90.2110273227%, Avg loss: 0.257393
Epoch 218
-------------------------------
loss: 0.370192  [    0/ 5994]
loss: 0.363038  [ 1600/ 5994]
loss: 0.365410  [ 3200/ 5994]
loss: 0.357261  [ 4800/ 5994]
Train Accuracy: 90.2851569518%, Avg loss: 0.255275
Train Accuracy: 90.2205420284%, Avg loss: 0.257234
Epoch 219
-------------------------------
loss: 0.349518  [    0/ 5994]
loss: 0.383966  [ 1600/ 5994]
loss: 0.354380  [ 3200/ 5994]
loss: 0.360740  [ 4800/ 5994]
Train Accuracy: 90.2897555675%, Avg loss: 0.255254
Train Accuracy: 90.2155080853%, Avg loss: 0.257237
Epoch 220
-------------------------------
loss: 0.371412  [    0/ 5994]
loss: 0.351917  [ 1600/ 5994]
loss: 0.372607  [ 3200/ 5994]
loss: 0.341348  [ 4800/ 5994]
Train Accuracy: 90.2860125082%, Avg loss: 0.255262
Train Accuracy: 90.2144017242%, Avg loss: 0.257325
Epoch 221
-------------------------------
loss: 0.362029  [    0/ 5994]
loss: 0.370999  [ 1600/ 5994]
loss: 0.376189  [ 3200/ 5994]
loss: 0.387557  [ 4800/ 5994]
Train Accuracy: 90.2863333419%, Avg loss: 0.255345
Train Accuracy: 90.2114145491%, Avg loss: 0.257367
Saved PyTorch Model State to model.pth
Epoch 222
-------------------------------
loss: 0.342491  [    0/ 5994]
loss: 0.350350  [ 1600/ 5994]
loss: 0.360056  [ 3200/ 5994]
loss: 0.369902  [ 4800/ 5994]
Train Accuracy: 90.2855312578%, Avg loss: 0.255171
Train Accuracy: 90.2152868131%, Avg loss: 0.257211
Epoch 223
-------------------------------
loss: 0.376794  [    0/ 5994]
loss: 0.374810  [ 1600/ 5994]
loss: 0.348383  [ 3200/ 5994]
loss: 0.339286  [ 4800/ 5994]
Train Accuracy: 90.2872958429%, Avg loss: 0.255221
Train Accuracy: 90.2170016728%, Avg loss: 0.257238
Epoch 224
-------------------------------
loss: 0.369594  [    0/ 5994]
loss: 0.417458  [ 1600/ 5994]
loss: 0.364153  [ 3200/ 5994]
loss: 0.356379  [ 4800/ 5994]
Train Accuracy: 90.2881513993%, Avg loss: 0.255221
Train Accuracy: 90.2153974492%, Avg loss: 0.257227
Epoch 225
-------------------------------
loss: 0.401554  [    0/ 5994]
loss: 0.400008  [ 1600/ 5994]
loss: 0.343095  [ 3200/ 5994]
loss: 0.380309  [ 4800/ 5994]
Train Accuracy: 90.2883652884%, Avg loss: 0.255164
Train Accuracy: 90.2139591797%, Avg loss: 0.257203
Epoch 226
-------------------------------
loss: 0.373105  [    0/ 5994]
loss: 0.380875  [ 1600/ 5994]
loss: 0.386286  [ 3200/ 5994]
loss: 0.350171  [ 4800/ 5994]
Train Accuracy: 90.2858520914%, Avg loss: 0.255221
Train Accuracy: 90.2154527672%, Avg loss: 0.257235
Saved PyTorch Model State to model.pth
Epoch 227
-------------------------------
loss: 0.392845  [    0/ 5994]
loss: 0.364061  [ 1600/ 5994]
loss: 0.373467  [ 3200/ 5994]
loss: 0.355837  [ 4800/ 5994]
Train Accuracy: 90.2884722329%, Avg loss: 0.255166
Train Accuracy: 90.2122996380%, Avg loss: 0.257214
Epoch 228
-------------------------------
loss: 0.374648  [    0/ 5994]
loss: 0.368647  [ 1600/ 5994]
loss: 0.344355  [ 3200/ 5994]
loss: 0.411684  [ 4800/ 5994]
Train Accuracy: 90.2852104241%, Avg loss: 0.255207
Train Accuracy: 90.2143464061%, Avg loss: 0.257243
Epoch 229
-------------------------------
loss: 0.353717  [    0/ 5994]
loss: 0.359641  [ 1600/ 5994]
loss: 0.378541  [ 3200/ 5994]
loss: 0.341011  [ 4800/ 5994]
Train Accuracy: 90.2883118161%, Avg loss: 0.255098
Train Accuracy: 90.2120230477%, Avg loss: 0.257208
Epoch 230
-------------------------------
loss: 0.325599  [    0/ 5994]
loss: 0.358818  [ 1600/ 5994]
loss: 0.364378  [ 3200/ 5994]
loss: 0.368230  [ 4800/ 5994]
Train Accuracy: 90.2893277893%, Avg loss: 0.255114
Train Accuracy: 90.2174442173%, Avg loss: 0.257153
Epoch 231
-------------------------------
loss: 0.379459  [    0/ 5994]
loss: 0.351415  [ 1600/ 5994]
loss: 0.359644  [ 3200/ 5994]
loss: 0.384016  [ 4800/ 5994]
Train Accuracy: 90.2902368180%, Avg loss: 0.255130
Train Accuracy: 90.2146783144%, Avg loss: 0.257154
Saved PyTorch Model State to model.pth
Epoch 232
-------------------------------
loss: 0.369496  [    0/ 5994]
loss: 0.362158  [ 1600/ 5994]
loss: 0.363368  [ 3200/ 5994]
loss: 0.353275  [ 4800/ 5994]
Train Accuracy: 90.2883652884%, Avg loss: 0.255089
Train Accuracy: 90.2161165839%, Avg loss: 0.257163
Epoch 233
-------------------------------
loss: 0.390263  [    0/ 5994]
loss: 0.381368  [ 1600/ 5994]
loss: 0.388410  [ 3200/ 5994]
loss: 0.346478  [ 4800/ 5994]
Train Accuracy: 90.2879909824%, Avg loss: 0.255050
Train Accuracy: 90.2123549561%, Avg loss: 0.257201
Epoch 234
-------------------------------
loss: 0.388941  [    0/ 5994]
loss: 0.426777  [ 1600/ 5994]
loss: 0.392734  [ 3200/ 5994]
loss: 0.336382  [ 4800/ 5994]
Train Accuracy: 90.2892208448%, Avg loss: 0.255110
Train Accuracy: 90.2149549047%, Avg loss: 0.257154
Epoch 235
-------------------------------
loss: 0.389491  [    0/ 5994]
loss: 0.367824  [ 1600/ 5994]
loss: 0.360537  [ 3200/ 5994]
loss: 0.368559  [ 4800/ 5994]
Train Accuracy: 90.2862798696%, Avg loss: 0.255002
Train Accuracy: 90.2164484923%, Avg loss: 0.257112
Epoch 236
-------------------------------
loss: 0.351053  [    0/ 5994]
loss: 0.371683  [ 1600/ 5994]
loss: 0.337929  [ 3200/ 5994]
loss: 0.357541  [ 4800/ 5994]
Train Accuracy: 90.2856382023%, Avg loss: 0.255038
Train Accuracy: 90.2125762283%, Avg loss: 0.257121
Saved PyTorch Model State to model.pth
Epoch 237
-------------------------------
loss: 0.406583  [    0/ 5994]
loss: 0.373464  [ 1600/ 5994]
loss: 0.362726  [ 3200/ 5994]
loss: 0.372461  [ 4800/ 5994]
Train Accuracy: 90.2879375102%, Avg loss: 0.255050
Train Accuracy: 90.2117464574%, Avg loss: 0.257136
Epoch 238
-------------------------------
loss: 0.385760  [    0/ 5994]
loss: 0.362766  [ 1600/ 5994]
loss: 0.383511  [ 3200/ 5994]
loss: 0.388794  [ 4800/ 5994]
Train Accuracy: 90.2883652884%, Avg loss: 0.254951
Train Accuracy: 90.2130740908%, Avg loss: 0.257138
Epoch 239
-------------------------------
loss: 0.365853  [    0/ 5994]
loss: 0.390748  [ 1600/ 5994]
loss: 0.359270  [ 3200/ 5994]
loss: 0.391240  [ 4800/ 5994]
Train Accuracy: 90.2902368180%, Avg loss: 0.254994
Train Accuracy: 90.2155634033%, Avg loss: 0.257079
Epoch 240
-------------------------------
loss: 0.366672  [    0/ 5994]
loss: 0.337300  [ 1600/ 5994]
loss: 0.383436  [ 3200/ 5994]
loss: 0.392492  [ 4800/ 5994]
Train Accuracy: 90.2851569518%, Avg loss: 0.254946
Train Accuracy: 90.2141804519%, Avg loss: 0.257109
Epoch 241
-------------------------------
loss: 0.373089  [    0/ 5994]
loss: 0.365519  [ 1600/ 5994]
loss: 0.378493  [ 3200/ 5994]
loss: 0.370725  [ 4800/ 5994]
Train Accuracy: 90.2916805695%, Avg loss: 0.254966
Train Accuracy: 90.2188824867%, Avg loss: 0.257117
Saved PyTorch Model State to model.pth
Epoch 242
-------------------------------
loss: 0.344416  [    0/ 5994]
loss: 0.340496  [ 1600/ 5994]
loss: 0.363171  [ 3200/ 5994]
loss: 0.375856  [ 4800/ 5994]
Train Accuracy: 90.2878305656%, Avg loss: 0.254945
Train Accuracy: 90.2136825894%, Avg loss: 0.257086
Epoch 243
-------------------------------
loss: 0.351633  [    0/ 5994]
loss: 0.394020  [ 1600/ 5994]
loss: 0.345273  [ 3200/ 5994]
loss: 0.357332  [ 4800/ 5994]
Train Accuracy: 90.2867076478%, Avg loss: 0.254956
Train Accuracy: 90.2143464061%, Avg loss: 0.257117
Epoch 244
-------------------------------
loss: 0.343373  [    0/ 5994]
loss: 0.345723  [ 1600/ 5994]
loss: 0.353768  [ 3200/ 5994]
loss: 0.356256  [ 4800/ 5994]
Train Accuracy: 90.2906645962%, Avg loss: 0.254884
Train Accuracy: 90.2141251339%, Avg loss: 0.257088
Epoch 245
-------------------------------
loss: 0.379788  [    0/ 5994]
loss: 0.336839  [ 1600/ 5994]
loss: 0.363975  [ 3200/ 5994]
loss: 0.377935  [ 4800/ 5994]
Train Accuracy: 90.2875632042%, Avg loss: 0.254909
Train Accuracy: 90.2130187727%, Avg loss: 0.257110
Epoch 246
-------------------------------
loss: 0.389495  [    0/ 5994]
loss: 0.384577  [ 1600/ 5994]
loss: 0.412566  [ 3200/ 5994]
loss: 0.363652  [ 4800/ 5994]
Train Accuracy: 90.2876166765%, Avg loss: 0.254971
Train Accuracy: 90.2138485436%, Avg loss: 0.257168
Saved PyTorch Model State to model.pth
Epoch 247
-------------------------------
loss: 0.382722  [    0/ 5994]
loss: 0.370089  [ 1600/ 5994]
loss: 0.361961  [ 3200/ 5994]
loss: 0.378606  [ 4800/ 5994]
Train Accuracy: 90.2882583438%, Avg loss: 0.254896
Train Accuracy: 90.2118570935%, Avg loss: 0.257069
Epoch 248
-------------------------------
loss: 0.356199  [    0/ 5994]
loss: 0.359083  [ 1600/ 5994]
loss: 0.397576  [ 3200/ 5994]
loss: 0.359236  [ 4800/ 5994]
Train Accuracy: 90.2875632042%, Avg loss: 0.254963
Train Accuracy: 90.2137379075%, Avg loss: 0.257072
Epoch 249
-------------------------------
loss: 0.360428  [    0/ 5994]
loss: 0.369379  [ 1600/ 5994]
loss: 0.360487  [ 3200/ 5994]
loss: 0.374537  [ 4800/ 5994]
Train Accuracy: 90.2857986191%, Avg loss: 0.254909
Train Accuracy: 90.2160612659%, Avg loss: 0.257111
Epoch 250
-------------------------------
loss: 0.353010  [    0/ 5994]
loss: 0.367002  [ 1600/ 5994]
loss: 0.353335  [ 3200/ 5994]
loss: 0.350913  [ 4800/ 5994]
Train Accuracy: 90.2826437549%, Avg loss: 0.254992
Train Accuracy: 90.2115251852%, Avg loss: 0.257173
Epoch 251
-------------------------------
loss: 0.338782  [    0/ 5994]
loss: 0.377716  [ 1600/ 5994]
loss: 0.383757  [ 3200/ 5994]
loss: 0.326855  [ 4800/ 5994]
Train Accuracy: 90.2875097320%, Avg loss: 0.254869
Train Accuracy: 90.2132953630%, Avg loss: 0.257059
Saved PyTorch Model State to model.pth
Epoch 252
-------------------------------
loss: 0.346338  [    0/ 5994]
loss: 0.388686  [ 1600/ 5994]
loss: 0.372702  [ 3200/ 5994]
loss: 0.348254  [ 4800/ 5994]
Train Accuracy: 90.2858520914%, Avg loss: 0.254898
Train Accuracy: 90.2129634547%, Avg loss: 0.257119
Epoch 253
-------------------------------
loss: 0.335297  [    0/ 5994]
loss: 0.371446  [ 1600/ 5994]
loss: 0.371630  [ 3200/ 5994]
loss: 0.387620  [ 4800/ 5994]
Train Accuracy: 90.2879375102%, Avg loss: 0.254903
Train Accuracy: 90.2126868644%, Avg loss: 0.257063
Epoch 254
-------------------------------
loss: 0.369733  [    0/ 5994]
loss: 0.342447  [ 1600/ 5994]
loss: 0.339544  [ 3200/ 5994]
loss: 0.364875  [ 4800/ 5994]
Train Accuracy: 90.2893277893%, Avg loss: 0.254848
Train Accuracy: 90.2130740908%, Avg loss: 0.257054
Epoch 255
-------------------------------
loss: 0.360064  [    0/ 5994]
loss: 0.361928  [ 1600/ 5994]
loss: 0.342576  [ 3200/ 5994]
loss: 0.350391  [ 4800/ 5994]
Train Accuracy: 90.2862263973%, Avg loss: 0.254858
Train Accuracy: 90.2108060505%, Avg loss: 0.257088
Epoch 256
-------------------------------
loss: 0.372079  [    0/ 5994]
loss: 0.367624  [ 1600/ 5994]
loss: 0.359762  [ 3200/ 5994]
loss: 0.364777  [ 4800/ 5994]
Train Accuracy: 90.2903972348%, Avg loss: 0.254758
Train Accuracy: 90.2131294088%, Avg loss: 0.257020
Saved PyTorch Model State to model.pth
Epoch 257
-------------------------------
loss: 0.384224  [    0/ 5994]
loss: 0.357568  [ 1600/ 5994]
loss: 0.382666  [ 3200/ 5994]
loss: 0.374013  [ 4800/ 5994]
Train Accuracy: 90.2909854299%, Avg loss: 0.254817
Train Accuracy: 90.2120230477%, Avg loss: 0.257040
Epoch 258
-------------------------------
loss: 0.383698  [    0/ 5994]
loss: 0.352347  [ 1600/ 5994]
loss: 0.356934  [ 3200/ 5994]
loss: 0.368977  [ 4800/ 5994]
Train Accuracy: 90.2891139002%, Avg loss: 0.254799
Train Accuracy: 90.2101975518%, Avg loss: 0.257116
Epoch 259
-------------------------------
loss: 0.373635  [    0/ 5994]
loss: 0.363794  [ 1600/ 5994]
loss: 0.373853  [ 3200/ 5994]
loss: 0.355505  [ 4800/ 5994]
Train Accuracy: 90.2872958429%, Avg loss: 0.254735
Train Accuracy: 90.2117464574%, Avg loss: 0.257030
Epoch 260
-------------------------------
loss: 0.412864  [    0/ 5994]
loss: 0.374853  [ 1600/ 5994]
loss: 0.383417  [ 3200/ 5994]
loss: 0.351359  [ 4800/ 5994]
Train Accuracy: 90.2876166765%, Avg loss: 0.254780
Train Accuracy: 90.2106400963%, Avg loss: 0.257112
Epoch 261
-------------------------------
loss: 0.380855  [    0/ 5994]
loss: 0.347735  [ 1600/ 5994]
loss: 0.417534  [ 3200/ 5994]
loss: 0.343458  [ 4800/ 5994]
Train Accuracy: 90.2895951507%, Avg loss: 0.254780
Train Accuracy: 90.2112485949%, Avg loss: 0.257090
Saved PyTorch Model State to model.pth
Epoch 262
-------------------------------
loss: 0.332889  [    0/ 5994]
loss: 0.359257  [ 1600/ 5994]
loss: 0.389648  [ 3200/ 5994]
loss: 0.341843  [ 4800/ 5994]
Train Accuracy: 90.2910389022%, Avg loss: 0.254765
Train Accuracy: 90.2120783658%, Avg loss: 0.257006
Epoch 263
-------------------------------
loss: 0.380242  [    0/ 5994]
loss: 0.377588  [ 1600/ 5994]
loss: 0.357142  [ 3200/ 5994]
loss: 0.386423  [ 4800/ 5994]
Train Accuracy: 90.2865472310%, Avg loss: 0.254775
Train Accuracy: 90.2118570935%, Avg loss: 0.257017
Epoch 264
-------------------------------
loss: 0.351084  [    0/ 5994]
loss: 0.359300  [ 1600/ 5994]
loss: 0.352144  [ 3200/ 5994]
loss: 0.382454  [ 4800/ 5994]
Train Accuracy: 90.2900229289%, Avg loss: 0.254704
Train Accuracy: 90.2100869157%, Avg loss: 0.257032
Epoch 265
-------------------------------
loss: 0.339897  [    0/ 5994]
loss: 0.352782  [ 1600/ 5994]
loss: 0.373210  [ 3200/ 5994]
loss: 0.367306  [ 4800/ 5994]
Train Accuracy: 90.2925895981%, Avg loss: 0.254723
Train Accuracy: 90.2128528186%, Avg loss: 0.257001
Epoch 266
-------------------------------
loss: 0.350798  [    0/ 5994]
loss: 0.377893  [ 1600/ 5994]
loss: 0.384523  [ 3200/ 5994]
loss: 0.414542  [ 4800/ 5994]
Train Accuracy: 90.2957444624%, Avg loss: 0.254716
Train Accuracy: 90.2129634547%, Avg loss: 0.256991
Saved PyTorch Model State to model.pth
Epoch 267
-------------------------------
loss: 0.333029  [    0/ 5994]
loss: 0.329295  [ 1600/ 5994]
loss: 0.391198  [ 3200/ 5994]
loss: 0.360084  [ 4800/ 5994]
Train Accuracy: 90.2902368180%, Avg loss: 0.254710
Train Accuracy: 90.2119124116%, Avg loss: 0.257086
Epoch 268
-------------------------------
loss: 0.352745  [    0/ 5994]
loss: 0.400589  [ 1600/ 5994]
loss: 0.357754  [ 3200/ 5994]
loss: 0.342807  [ 4800/ 5994]
Train Accuracy: 90.2913597358%, Avg loss: 0.254719
Train Accuracy: 90.2091465088%, Avg loss: 0.257066
Epoch 269
-------------------------------
loss: 0.375928  [    0/ 5994]
loss: 0.354906  [ 1600/ 5994]
loss: 0.388231  [ 3200/ 5994]
loss: 0.388992  [ 4800/ 5994]
Train Accuracy: 90.2891673725%, Avg loss: 0.254734
Train Accuracy: 90.2099209616%, Avg loss: 0.257054
Epoch 270
-------------------------------
loss: 0.369396  [    0/ 5994]
loss: 0.379242  [ 1600/ 5994]
loss: 0.378877  [ 3200/ 5994]
loss: 0.349590  [ 4800/ 5994]
Train Accuracy: 90.2908784853%, Avg loss: 0.254730
Train Accuracy: 90.2119124116%, Avg loss: 0.257022
Epoch 271
-------------------------------
loss: 0.351297  [    0/ 5994]
loss: 0.345715  [ 1600/ 5994]
loss: 0.353185  [ 3200/ 5994]
loss: 0.410766  [ 4800/ 5994]
Train Accuracy: 90.2900229289%, Avg loss: 0.254703
Train Accuracy: 90.2139038616%, Avg loss: 0.257018
Saved PyTorch Model State to model.pth
Epoch 272
-------------------------------
loss: 0.412393  [    0/ 5994]
loss: 0.376690  [ 1600/ 5994]
loss: 0.404888  [ 3200/ 5994]
loss: 0.342386  [ 4800/ 5994]
Train Accuracy: 90.2870284815%, Avg loss: 0.254663
Train Accuracy: 90.2114698671%, Avg loss: 0.256989
Epoch 273
-------------------------------
loss: 0.370182  [    0/ 5994]
loss: 0.327271  [ 1600/ 5994]
loss: 0.359450  [ 3200/ 5994]
loss: 0.385860  [ 4800/ 5994]
Train Accuracy: 90.2919479308%, Avg loss: 0.254641
Train Accuracy: 90.2117464574%, Avg loss: 0.256958
Epoch 274
-------------------------------
loss: 0.348108  [    0/ 5994]
loss: 0.358608  [ 1600/ 5994]
loss: 0.370334  [ 3200/ 5994]
loss: 0.396844  [ 4800/ 5994]
Train Accuracy: 90.2898090398%, Avg loss: 0.254633
Train Accuracy: 90.2134613172%, Avg loss: 0.256972
Epoch 275
-------------------------------
loss: 0.343465  [    0/ 5994]
loss: 0.364869  [ 1600/ 5994]
loss: 0.364034  [ 3200/ 5994]
loss: 0.372826  [ 4800/ 5994]
Train Accuracy: 90.2915201526%, Avg loss: 0.254647
Train Accuracy: 90.2129634547%, Avg loss: 0.256956
Epoch 276
-------------------------------
loss: 0.339680  [    0/ 5994]
loss: 0.331704  [ 1600/ 5994]
loss: 0.351622  [ 3200/ 5994]
loss: 0.378473  [ 4800/ 5994]
Train Accuracy: 90.2911458467%, Avg loss: 0.254588
Train Accuracy: 90.2118017755%, Avg loss: 0.256996
Saved PyTorch Model State to model.pth
Epoch 277
-------------------------------
loss: 0.381746  [    0/ 5994]
loss: 0.319805  [ 1600/ 5994]
loss: 0.370658  [ 3200/ 5994]
loss: 0.345822  [ 4800/ 5994]
Train Accuracy: 90.2934451546%, Avg loss: 0.254586
Train Accuracy: 90.2129634547%, Avg loss: 0.256993
Epoch 278
-------------------------------
loss: 0.382575  [    0/ 5994]
loss: 0.371119  [ 1600/ 5994]
loss: 0.362171  [ 3200/ 5994]
loss: 0.355710  [ 4800/ 5994]
Train Accuracy: 90.2912527913%, Avg loss: 0.254634
Train Accuracy: 90.2147889506%, Avg loss: 0.256974
Epoch 279
-------------------------------
loss: 0.347157  [    0/ 5994]
loss: 0.365149  [ 1600/ 5994]
loss: 0.358095  [ 3200/ 5994]
loss: 0.389542  [ 4800/ 5994]
Train Accuracy: 90.2925895981%, Avg loss: 0.254571
Train Accuracy: 90.2113592310%, Avg loss: 0.257003
Epoch 280
-------------------------------
loss: 0.381775  [    0/ 5994]
loss: 0.338156  [ 1600/ 5994]
loss: 0.352693  [ 3200/ 5994]
loss: 0.392523  [ 4800/ 5994]
Train Accuracy: 90.2921083477%, Avg loss: 0.254569
Train Accuracy: 90.2098103254%, Avg loss: 0.256973
Epoch 281
-------------------------------
loss: 0.342937  [    0/ 5994]
loss: 0.382170  [ 1600/ 5994]
loss: 0.375213  [ 3200/ 5994]
loss: 0.362342  [ 4800/ 5994]
Train Accuracy: 90.2856916746%, Avg loss: 0.254532
Train Accuracy: 90.2106954144%, Avg loss: 0.256967
Saved PyTorch Model State to model.pth
Epoch 282
-------------------------------
loss: 0.363356  [    0/ 5994]
loss: 0.392343  [ 1600/ 5994]
loss: 0.437208  [ 3200/ 5994]
loss: 0.378768  [ 4800/ 5994]
Train Accuracy: 90.2866541755%, Avg loss: 0.254555
Train Accuracy: 90.2122996380%, Avg loss: 0.257010
Epoch 283
-------------------------------
loss: 0.342392  [    0/ 5994]
loss: 0.355876  [ 1600/ 5994]
loss: 0.380484  [ 3200/ 5994]
loss: 0.388219  [ 4800/ 5994]
Train Accuracy: 90.2889000111%, Avg loss: 0.254562
Train Accuracy: 90.2100869157%, Avg loss: 0.256957
Epoch 284
-------------------------------
loss: 0.340939  [    0/ 5994]
loss: 0.339187  [ 1600/ 5994]
loss: 0.364824  [ 3200/ 5994]
loss: 0.370040  [ 4800/ 5994]
Train Accuracy: 90.2895951507%, Avg loss: 0.254584
Train Accuracy: 90.2099209616%, Avg loss: 0.256944
Epoch 285
-------------------------------
loss: 0.344533  [    0/ 5994]
loss: 0.359260  [ 1600/ 5994]
loss: 0.376690  [ 3200/ 5994]
loss: 0.349816  [ 4800/ 5994]
Train Accuracy: 90.2902368180%, Avg loss: 0.254583
Train Accuracy: 90.2098656435%, Avg loss: 0.256967
Epoch 286
-------------------------------
loss: 0.374532  [    0/ 5994]
loss: 0.349624  [ 1600/ 5994]
loss: 0.351420  [ 3200/ 5994]
loss: 0.366901  [ 4800/ 5994]
Train Accuracy: 90.2908250130%, Avg loss: 0.254520
Train Accuracy: 90.2127975005%, Avg loss: 0.256958
Saved PyTorch Model State to model.pth
Epoch 287
-------------------------------
loss: 0.329438  [    0/ 5994]
loss: 0.335741  [ 1600/ 5994]
loss: 0.367274  [ 3200/ 5994]
loss: 0.361691  [ 4800/ 5994]
Train Accuracy: 90.2866007033%, Avg loss: 0.254548
Train Accuracy: 90.2130187727%, Avg loss: 0.256959
Epoch 288
-------------------------------
loss: 0.408642  [    0/ 5994]
loss: 0.383344  [ 1600/ 5994]
loss: 0.363488  [ 3200/ 5994]
loss: 0.385229  [ 4800/ 5994]
Train Accuracy: 90.2909319576%, Avg loss: 0.254482
Train Accuracy: 90.2121336838%, Avg loss: 0.256946
Epoch 289
-------------------------------
loss: 0.326276  [    0/ 5994]
loss: 0.378962  [ 1600/ 5994]
loss: 0.385093  [ 3200/ 5994]
loss: 0.354697  [ 4800/ 5994]
Train Accuracy: 90.2922687645%, Avg loss: 0.254527
Train Accuracy: 90.2126315463%, Avg loss: 0.256982
Epoch 290
-------------------------------
loss: 0.357778  [    0/ 5994]
loss: 0.337469  [ 1600/ 5994]
loss: 0.407065  [ 3200/ 5994]
loss: 0.361947  [ 4800/ 5994]
Train Accuracy: 90.2907715408%, Avg loss: 0.254495
Train Accuracy: 90.2109166866%, Avg loss: 0.256957
Epoch 291
-------------------------------
loss: 0.337472  [    0/ 5994]
loss: 0.366758  [ 1600/ 5994]
loss: 0.380130  [ 3200/ 5994]
loss: 0.355477  [ 4800/ 5994]
Train Accuracy: 90.2914132081%, Avg loss: 0.254460
Train Accuracy: 90.2120783658%, Avg loss: 0.256947
Saved PyTorch Model State to model.pth
Epoch 292
-------------------------------
loss: 0.344195  [    0/ 5994]
loss: 0.388815  [ 1600/ 5994]
loss: 0.391495  [ 3200/ 5994]
loss: 0.370230  [ 4800/ 5994]
Train Accuracy: 90.2938729328%, Avg loss: 0.254450
Train Accuracy: 90.2142357700%, Avg loss: 0.256900
Epoch 293
-------------------------------
loss: 0.335739  [    0/ 5994]
loss: 0.361172  [ 1600/ 5994]
loss: 0.356905  [ 3200/ 5994]
loss: 0.344261  [ 4800/ 5994]
Train Accuracy: 90.2939264050%, Avg loss: 0.254501
Train Accuracy: 90.2150102228%, Avg loss: 0.256960
Epoch 294
-------------------------------
loss: 0.375182  [    0/ 5994]
loss: 0.342664  [ 1600/ 5994]
loss: 0.373385  [ 3200/ 5994]
loss: 0.357819  [ 4800/ 5994]
Train Accuracy: 90.2927500150%, Avg loss: 0.254497
Train Accuracy: 90.2128528186%, Avg loss: 0.257043
Epoch 295
-------------------------------
loss: 0.354535  [    0/ 5994]
loss: 0.422036  [ 1600/ 5994]
loss: 0.353041  [ 3200/ 5994]
loss: 0.373576  [ 4800/ 5994]
Train Accuracy: 90.2931777932%, Avg loss: 0.254426
Train Accuracy: 90.2142910880%, Avg loss: 0.256947
Epoch 296
-------------------------------
loss: 0.378410  [    0/ 5994]
loss: 0.338475  [ 1600/ 5994]
loss: 0.432619  [ 3200/ 5994]
loss: 0.394608  [ 4800/ 5994]
Train Accuracy: 90.2898625121%, Avg loss: 0.254415
Train Accuracy: 90.2130740908%, Avg loss: 0.256914
Saved PyTorch Model State to model.pth
Epoch 297
-------------------------------
loss: 0.338672  [    0/ 5994]

```